# Set-up

In [ ]:
import os

script_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.dirname(script_dir)

## Importing modules

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
import cellhint
import harmonypy as hm
import seaborn as sns
import random
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Loading custom scripts

In [ ]:
def assign_labels(dataset, reduction, n_neighbors, label_input, label_output, frequency_threshold):
    # Compute the neighborhood graph
    sc.pp.neighbors(dataset, use_rep=reduction, n_neighbors=n_neighbors)

    # Perform the clustering
    sc.tl.leiden(dataset, key_added='clusters', resolution=10)

    # Initialize the new column with the existing labels
    dataset.obs[label_output] = dataset.obs[label_input]

    # For each cluster, find the most frequent label and assign it to all cells in the cluster
    for cluster in dataset.obs['clusters'].unique():
        cluster_labels = dataset.obs.loc[dataset.obs['clusters'] == cluster, label_input]
        most_frequent_label = cluster_labels.mode()[0]
        frequency = (cluster_labels == most_frequent_label).mean()

        if frequency > frequency_threshold:
            dataset.obs.loc[dataset.obs['clusters'] == cluster, label_output] = most_frequent_label

    return dataset

In [ ]:
def grouped_obs_mean(adata, group_key, layer=None, gene_symbols=None):
    if layer is not None:
        getX = lambda x: x.layers[layer]
    else:
        getX = lambda x: x.X
    if gene_symbols is not None:
        new_idx = adata.var[idx]
    else:
        new_idx = adata.var_names

    grouped = adata.obs.groupby(group_key)
    out = pd.DataFrame(
        np.zeros((adata.shape[1], len(grouped)), dtype=np.float64),
        columns=list(grouped.groups.keys()),
        index=adata.var_names
    )

    for group, idx in grouped.indices.items():
        X = getX(adata[idx])
        out[group] = np.ravel(X.mean(axis=0, dtype=np.float64))
    return out

In [ ]:
import sys
sys.path.append(parent_dir + '/Scripts/SingleCellProcessing')

import SCUtils

In [ ]:
pip list

PYTHONHASHSEED was set as envinronmental variable to 0 as follows:
    
conda env config vars set PYTHONHASHSEED=0

In [ ]:
os.environ['PYTHONHASHSEED'] = '0'
random.seed(42)
np.random.seed(42)

In [ ]:
def ensure_pythonhashseed(seed=0):
    current_seed = os.environ.get("PYTHONHASHSEED")

    seed = str(seed)
    if current_seed is None or current_seed != seed:
        print(f'Setting PYTHONHASHSEED="{seed}"')
        os.environ["PYTHONHASHSEED"] = seed
        # restart the current process
        os.execl(sys.executable, sys.executable, *sys.argv)

In [ ]:
import random

hash = random.getrandbits(128)

print("hash value: %032x" % hash)

## Defining data path

In [ ]:
# Specify the folder path
data_path = parent_dir + "/Data"
figures_path = parent_dir + "/Figures"

# Processing Zhang X. et al. (2024) dataset

In [ ]:
Zhang_dataset = sc.read_h5ad(data_path + '/References/Zhang' + '/adata_combined_rna_adt_annotated-titrated.h5ad')

### Dataset Description

In [ ]:
Zhang_dataset

In [ ]:
Zhang_dataset.obsm['X_umap'] = Zhang_dataset.obsm['X_umap'].values
Zhang_dataset.obsm['X_umap-titration'] = Zhang_dataset.obsm['X_umap-titration'].values

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Zhang_dataset, 
                color='Level 3 Multimodal', 
                basis='X_umap', 
                legend_loc='right margin', 
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(4.5, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Zhang X. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Original annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Place the legend below the plot
legend = ax.legend(loc='upper center', 
                   bbox_to_anchor=(0.5, -0.05),
                   prop={'size': 4.8},
                   ncol=5)

# Reduce the size of the dots in the legend
for handle in legend.legend_handles:
    handle._sizes = [10]

# Adjust the layout to make room for the legend
plt.subplots_adjust(bottom=0.3)

# Save the figure at 300 dpi
plt.savefig(figures_path + "/ZhangX_original_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
vars_to_keep = [
    'CD230', 'Hu.C5L2', 'Hu.CD10', 'Hu.CD101', 'Hu.CD102', 'Hu.CD103', 'Hu.CD105_43A3', 'Hu.CD106', 'Hu.CD109', 'Hu.CD110', 
    'Hu.CD112', 'Hu.CD115', 'Hu.CD116', 'Hu.CD117', 'Hu.CD119', 'Hu.CD11a', 'Hu.CD11b', 'Hu.CD11c', 'Hu.CD123', 'Hu.CD127', 
    'Hu.CD13', 'Hu.CD133_S16016B', 'Hu.CD135', 'Hu.CD138_DL.101', 'Hu.CD140b', 'Hu.CD141', 'Hu.CD14_M5E2', 'Hu.CD150', 'Hu.CD151', 
    'Hu.CD154', 'Hu.CD155', 'Hu.CD158e1', 'Hu.CD158f', 'Hu.CD15_W6D3', 'Hu.CD16', 'Hu.CD162', 'Hu.CD163', 'Hu.CD164', 'Hu.CD172a', 
    'Hu.CD177', 'Hu.CD18', 'Hu.CD183', 'Hu.CD185', 'Hu.CD186', 'Hu.CD19', 'Hu.CD192', 'Hu.CD1a', 'Hu.CD1d', 'Hu.CD2', 'Hu.CD200', 
    'Hu.CD201', 'Hu.CD202b', 'Hu.CD205', 'Hu.CD226_TX25', 'Hu.CD235a', 'Hu.CD24', 'Hu.CD25', 'Hu.CD26', 'Hu.CD27', 'Hu.CD271', 
    'Hu.CD274', 'Hu.CD279', 'Hu.CD28', 'Hu.CD29', 'Hu.CD304', 'Hu.CD305_LAIR1', 'Hu.CD309', 'Hu.CD32', 'Hu.CD325', 'Hu.CD326', 
    'Hu.CD33', 'Hu.CD335', 'Hu.CD34', 'Hu.CD35', 'Hu.CD354', 'Hu.CD36', 'Hu.CD366', 'Hu.CD37', 'Hu.CD38_HIT2', 'Hu.CD41', 'Hu.CD42b', 
    'Hu.CD43', 'Hu.CD45RA', 'Hu.CD45RB', 'Hu.CD45RO', 'Hu.CD45_2D1', 'Hu.CD47', 'Hu.CD49b', 'Hu.CD4_RPA.T4', 'Hu.CD5', 'Hu.CD52', 
    'Hu.CD54', 'Hu.CD55', 'Hu.CD56', 'Hu.CD57', 'Hu.CD58', 'Hu.CD59', 'Hu.CD61', 'Hu.CD62L', 'Hu.CD62P', 'Hu.CD63', 'Hu.CD64', 
    'Hu.CD69', 'Hu.CD7', 'Hu.CD71', 'Hu.CD72', 'Hu.CD73', 'Hu.CD8', 'Hu.CD81', 'Hu.CD82', 'Hu.CD83', 'Hu.CD84', 'Hu.CD85g', 'Hu.CD9', 
    'Hu.CD90', 'Hu.CD93', 'Hu.CD98', 'Hu.CLEC1B', 'Hu.Cadherin.11', 'Hu.FR.b', 'Hu.FceRIa', 'Hu.GARP', 'Hu.GPR56', 'Hu.Galectin.9', 
    'Hu.HLA.ABC', 'Hu.HLA.DR.DP.DQ', 'Hu.KLRG1', 'Hu.TIM.4', 'Hu.TSPAN33', 'HuMs.CD44', 'HuMs.CD49f', 'HuMs.integrin.b7', 
    'Isotype_G0114F7', 'Isotype_HTK888', 'Isotype_MOPC.173', 'Isotype_MOPC.21', 'Isotype_MPC.11', 'Isotype_RTK2071', 'Isotype_RTK2758', 
    'Isotype_RTK4174', 'Isotype_RTK4530', 'Hu.IgG.Fc'
]

vars_to_keep = np.in1d(Zhang_dataset.var_names, vars_to_keep)
Zhang_dataset = Zhang_dataset[:, vars_to_keep]

In [ ]:
def zhang_dataset_adt_rename(dataset):
    dataset.var.rename(index=lambda x: x.replace('Hu.', '').replace('HuMs.', ''), inplace=True)
    dataset.var.rename(index={'FceRIa': 'FcεRIα'}, inplace=True)
    dataset.var.rename(index={'CD4_RPA.T4': 'CD4'}, inplace=True)
    dataset.var.rename(index={'CD45_2D1': 'CD45'}, inplace=True)
    dataset.var.rename(index={'CD38_HIT2': 'CD38'}, inplace=True)
    dataset.var.rename(index={'CD305_LAIR1': 'CD305'}, inplace=True)
    dataset.var.rename(index={'CD226_TX25': 'CD226'}, inplace=True)
    dataset.var.rename(index={'CD15_W6D3': 'CD15'}, inplace=True)
    dataset.var.rename(index={'CD14_M5E2': 'CD14'}, inplace=True)
    dataset.var.rename(index={'CD138_DL.101': 'CD138'}, inplace=True)
    dataset.var.rename(index={'CD133_S16016B': 'CD133'}, inplace=True)
    dataset.var.rename(index={'CD105_43A3': 'CD105'}, inplace=True)

zhang_dataset_adt_rename(Zhang_dataset)

In [ ]:
BF21_CD34 = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_BF21-CD34_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]
BF21_CD271 = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_BF21-CD271_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]
BF21_TNC = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_BF21-TNC_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]

WF26_CD34 = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_WF26-CD34_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]
WF26_CD271 = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_WF26-CD271_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]
WF26_TNC = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_WF26-TNC_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]

BM27_CD34 = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_BM27-CD34_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]
BM27_CD271 = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_BM27-CD271_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]
BM27_TNC = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_BM27-TNC_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]

WM34_CD34 = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_WM34-CD34_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]
WM34_CD271 = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_WM34-CD271_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]
WM34_TNC = sc.read_10x_h5(data_path + '/References/Zhang' + '/GSE245108_WM34-TNC_filtered_feature_bc_matrix.h5', gex_only=False)[:, vars_to_keep]

In [ ]:
zhang_dataset_adt_rename(BF21_CD34)
zhang_dataset_adt_rename(BF21_CD271)
zhang_dataset_adt_rename(BF21_TNC)

zhang_dataset_adt_rename(WF26_CD34)
zhang_dataset_adt_rename(WF26_CD271)
zhang_dataset_adt_rename(WF26_TNC)

zhang_dataset_adt_rename(BM27_CD34)
zhang_dataset_adt_rename(BM27_CD271)
zhang_dataset_adt_rename(BM27_TNC)

zhang_dataset_adt_rename(WM34_CD34)
zhang_dataset_adt_rename(WM34_CD271)
zhang_dataset_adt_rename(WM34_TNC)

In [ ]:
BF21_CD34.obs_names = BF21_CD34.obs_names + '.BF21_032123_CD34'
BF21_CD271.obs_names = BF21_CD271.obs_names + '.BF21_032123_CD271'
BF21_TNC.obs_names = BF21_TNC.obs_names + '.BF21_032123_TNC'

WF26_CD34.obs_names = WF26_CD34.obs_names + '.WF26_031423_CD34'
WF26_CD271.obs_names = WF26_CD271.obs_names + '.WF26_031423_CD271'
WF26_TNC.obs_names = WF26_TNC.obs_names + '.WF26_031423_TNC'

BM27_CD34.obs_names = BM27_CD34.obs_names + '.BM27_120522_CD34'
BM27_CD271.obs_names = BM27_CD271.obs_names + '.BM27_120522_CD271'
BM27_TNC.obs_names = BM27_TNC.obs_names + '.BM27_120522_TNC'

WM34_CD34.obs_names = WM34_CD34.obs_names + '.WM34_120522_CD34'
WM34_CD271.obs_names = WM34_CD271.obs_names + '.WM34_120522_CD271'
WM34_TNC.obs_names = WM34_TNC.obs_names + '.WM34_120522_TNC'

In [ ]:
WM34_CD34

In [ ]:
merged_adata = anndata.concat([BF21_CD34, BF21_CD271, BF21_TNC,
                               WF26_CD34, WF26_CD271, WF26_TNC,
                               BM27_CD34, BM27_CD271, BM27_TNC,
                               WM34_CD34, WM34_CD271, WM34_TNC], axis=0)

In [ ]:
obs_to_keep = np.in1d(merged_adata.obs_names, Zhang_dataset.obs_names)
merged_adata = merged_adata[obs_to_keep,:]
merged_adata = merged_adata[Zhang_dataset.obs_names]

In [ ]:
Zhang_dataset.X = merged_adata.X

In [ ]:
Zhang_dataset.obs['Batch'] = Zhang_dataset.obs['sample'].values

In [ ]:
Zhang_dataset.obs['Chemistry'] = 'BioLegend TotalSeqA'

# Loading Hao Y. et al. (2021) dataset

In [ ]:
Hao_dataset = sc.read_h5ad(data_path + "/References/Hao" + "/228AB_healthy_donors_PBMNCs.h5ad")

## Dataset Description

In [ ]:
Hao_dataset

In [ ]:
type(ax)

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Hao_dataset, 
                color='celltype.l3', 
                basis='X_wnn.umap', 
                legend_loc='right margin', 
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(4.5, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Hao Y. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Original annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Place the legend below the plot
legend = ax.legend(loc='upper center', 
                   bbox_to_anchor=(0.5, -0.05),
                   prop={'size': 4.8},
                   ncol=5)

# Reduce the size of the dots in the legend
for handle in legend.legend_handles:
    handle._sizes = [10]

# Adjust the layout to make room for the legend
plt.subplots_adjust(bottom=0.3)

# Save the figure at 300 dpi
plt.savefig(figures_path + "/HaoY_original_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
var_to_drop = np.in1d(Hao_dataset.var_names, SCUtils.Filter_duplicate_vars(Hao_dataset))
Hao_dataset = Hao_dataset[:, ~var_to_drop]

In [ ]:
Hao_dataset.obs['Chemistry'] = 'BioLegend TotalSeqA'

In [ ]:
Hao_dataset.obs['Batch'] = Hao_dataset.obs['donor'].values

# Loading Triana S. et al. (2021) dataset

In [ ]:
Triana_dataset = sc.read_h5ad(data_path + "/References/Triana" + "/97AB_young_and_old_adult_healthy_donor_BMMNCs.h5ad")

## Dataset Description

In [ ]:
Triana_dataset

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Triana_dataset, 
                color='CellTypes', 
                basis='X_mofaumap', 
                legend_loc='right margin', 
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(5.5, 5.5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Triana S. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Original annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Place the legend below the plot
legend = ax.legend(loc='upper center', 
                   bbox_to_anchor=(0.5, -0.05),
                   prop={'size': 4.8},
                   ncol=3)

# Reduce the size of the dots in the legend
for handle in legend.legend_handles:
    handle._sizes = [10]

# Adjust the layout to make room for the legend
plt.subplots_adjust(bottom=0.3)

# Save the figure at 300 dpi
plt.savefig(figures_path + "/TrianaS_original_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

Defining used chemistry as metadata to use for interdatasets integration

In [ ]:
Triana_dataset.obs['Chemistry'] = 'BD AbSeq'

Renaming feature labels to match across datasets

In [ ]:
Triana_dataset.var.rename(index={'HLA.DR': 'HLA-DR'}, inplace=True)
Triana_dataset.var.rename(index={'FCER1A': 'FcεRIα'}, inplace=True)

# Loading Luecken M.D. et al. (2021) dataset

In [ ]:
Luecken_dataset = sc.read_h5ad(data_path + "/References/Luecken" + "/140AB_adult_healthy_donor_BMMNCs.h5ad")

In [ ]:
adt = Luecken_dataset.var['feature_types'] == 'ADT'
Luecken_dataset = Luecken_dataset[:, adt]
Luecken_dataset.X = Luecken_dataset.layers['counts']

## Dataset Description

In [ ]:
Luecken_dataset

We are computing new embeddings as the original embeddings are not clear

In [ ]:
Luecken_dataset_normalized = SCUtils.Protein_normalization(Luecken_dataset.X)

In [ ]:
Luecken_dataset.obs['Batch'] = Luecken_dataset.obs['batch'].values

In [ ]:
ho = hm.run_harmony(Luecken_dataset_normalized, Luecken_dataset.obs, ["Batch"], 
                    max_iter_harmony = 30, random_state = 42)

In [ ]:
pc_std = np.std(ho.Z_corr, axis=1).tolist()

In [ ]:
sns.scatterplot(x=range(0, len(pc_std)), y=sorted(pc_std, reverse=True))

In [ ]:
Luecken_dataset.obsm["X_pcahm"] = ho.Z_corr.transpose()

In [ ]:
sc.pp.neighbors(Luecken_dataset, n_neighbors=30, n_pcs=50, use_rep="X_pcahm", random_state = 42)
sc.tl.umap(Luecken_dataset, random_state = 42)

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Luecken_dataset, 
                color='cell_type', 
                basis='X_umap', 
                legend_loc='right margin', 
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(5, 5.5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Luecken M.D. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Original annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Place the legend below the plot
legend = ax.legend(loc='upper center', 
                   bbox_to_anchor=(0.5, -0.05),
                   prop={'size': 4.8},
                   ncol=4)

# Reduce the size of the dots in the legend
for handle in legend.legend_handles:
    handle._sizes = [10]

# Adjust the layout to make room for the legend
plt.subplots_adjust(bottom=0.3)

# Save the figure at 300 dpi
plt.savefig(figures_path + "/LueckenMD_original_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
Luecken_dataset.obs['Chemistry'] = 'BioLegend TotalSeqB'

In [ ]:
Luecken_dataset.var.rename(index={'FceRIa': 'FcεRIα'}, inplace=True)

# Label harmonisation

### All cellular types

In [ ]:
original_labels = pd.Categorical(np.concatenate((Zhang_dataset.obs['Level 3 Multimodal'].values, 
                                                 Hao_dataset.obs['celltype.l3'].values, 
                                                 Triana_dataset.obs['CellTypes'].values, 
                                                 Luecken_dataset.obs['cell_type'].values)))

In [ ]:
adatas = {"Zhang": Zhang_dataset, 
          "Hao": Hao_dataset, 
          "Triana": Triana_dataset, 
          "Luecken": Luecken_dataset}

adatas = anndata.concat(adatas, 
                        label="dataset_name", 
                        join="outer")

In [ ]:
from venny4py.venny4py import *

# Create the Venn diagram with custom colors
sets = {'Zhang': set(list(Zhang_dataset.var_names)),
        'Hao': set(list(Hao_dataset.var_names)),
        'Triana': set(list(Triana_dataset.var_names)),
        'Luecken': set(list(Luecken_dataset.var_names))}

# Define custom colors for each dataset
colors = ['#1F77B4',  # Zhang - blue
          '#FE8010',  # Hao - orange  
          '#2EA02E',  # Triana - green
          '#D62828']  # Luecken - red

venny4py(sets=sets, out=figures_path, ext='png', colors=colors)

# Display the plot
plt.show()

# Specify the current file name and the new file name
current_file_name = figures_path + "/Venn_4.png"
new_file_name = figures_path + "/Shared_features_across_datasets.png"

# Rename the file
os.rename(current_file_name, new_file_name)

# Specify the current file name and the new file name
current_file_name = figures_path + "/Intersections_4.txt"
new_file_name = figures_path + "/Shared_features_across_datasets_list.txt"

# Rename the file
os.rename(current_file_name, new_file_name)

In [ ]:
common = SCUtils.Intersect_lists(Zhang_dataset.var_names, 
                                 Hao_dataset.var_names, 
                                 Triana_dataset.var_names, 
                                 Luecken_dataset.var_names)

In [ ]:
adatas = adatas[:, common]

In [ ]:
adatas.obs['Original_annotation'] = original_labels

In [ ]:
adatas.X = SCUtils.Protein_normalization(adatas.X)
sc.pp.regress_out(adatas, keys=['Chemistry'])

In [ ]:
random.seed(42)
np.random.seed(42)

ho = hm.run_harmony(adatas.X, adatas.obs, ["Batch"], 
                    max_iter_harmony = 30, random_state = 42)

In [ ]:
pc_std = np.std(ho.Z_corr, axis=1).tolist()

In [ ]:
sns.scatterplot(x=range(0, len(pc_std)), y=sorted(pc_std, reverse=True))

In [ ]:
adatas.obsm["X_pcahm"] = ho.Z_corr.transpose()

In [ ]:
sc.pp.neighbors(adatas, use_rep="X_pcahm", n_neighbors=30, metric='cosine', random_state = 42)

adatas.obsp["connectivities"] = np.round(adatas.obsp["connectivities"], decimals=5)
adatas.obsp["distances"] = np.round(adatas.obsp["distances"], decimals=5)

In [ ]:
sc.tl.umap(adatas, random_state = 42,  min_dist=0.3)

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='dataset_name', 
                basis='X_umap', 
                legend_loc='right margin', 
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.05)

# Place the legend below the plot
legend = ax.legend(loc='upper center', 
                   bbox_to_anchor=(0.9, 1),
                   prop={'size': 4.8},
                   ncol=1)

# Reduce the size of the dots in the legend
for handle in legend.legend_handles:
    handle._sizes = [10]

# Adjust the layout to make room for the legend
plt.subplots_adjust(bottom=0.3)

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_datasets_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Create subplot figure split by dataset_name
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

# Get unique datasets
datasets = adatas.obs['dataset_name'].unique()

for i, dataset in enumerate(datasets):
    # Filter data for current dataset
    dataset_mask = adatas.obs['dataset_name'] == dataset
    dataset_data = adatas[dataset_mask]
    
    # Plot UMAP for current dataset
    sc.pl.embedding(dataset_data, 
                    color='dataset_name', 
                    basis='X_umap', 
                    legend_loc='none',
                    add_outline=False,
                    frameon=False,
                    show=False,
                    ax=axes[i])
    
    # Set axis labels and title for each subplot
    axes[i].set_xlabel('UMAP 1', fontsize=12)
    axes[i].set_ylabel('UMAP 2', fontsize=12)
    axes[i].set_title(f'{dataset} dataset', fontsize=12, fontweight='bold')

# Remove empty subplot if odd number of datasets
if len(datasets) < 4:
    fig.delaxes(axes[3])

# Adjust layout
plt.tight_layout()

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_split_by_dataset.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
sc.tl.leiden(adatas, resolution=3.5, random_state = 42, 
             n_iterations=10)

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='leiden', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Cluster annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_leiden_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
alignment = cellhint.harmonize(adatas, 'dataset_name', 'Original_annotation', 
                               use_rep='X_pcahm', metric='cosine')

In [ ]:
cellhint.treeplot(alignment, save=figures_path + "/Merged_datasets_cellhint.png")

In [ ]:
adatas.obs[['low_hierarchy', 'high_hierarchy']] = alignment.reannotation.loc[adatas.obs_names, ['reannotation', 'group']]

In [ ]:
adatas.obs['low_hierarchy'] = pd.Categorical(adatas.obs['low_hierarchy'])
adatas.obs['high_hierarchy'] = pd.Categorical(adatas.obs['high_hierarchy'])

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='high_hierarchy', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Cluster annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# # Save the figure at 300 dpi
# plt.savefig(figures_path + "/Merged_datasets_leiden_annotation.png", 
#             dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='celltype.l2', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Cluster annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='leiden', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Cluster annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

# Create a crosstab to get the cell counts
crosstab = pd.crosstab(adatas.obs['high_hierarchy'], adatas.obs['leiden'])

# Prepare data for Sankey diagram
source_nodes = []
target_nodes = []
values = []
labels = []

# Add high_hierarchy labels
hierarchy_labels = list(crosstab.index)
leiden_labels = [f"Leiden_{x}" for x in crosstab.columns]
labels = hierarchy_labels + leiden_labels

# Create connections - only top 2 for each hierarchy
for i, hierarchy in enumerate(crosstab.index):
    # Get top 2 leiden clusters for this hierarchy
    hierarchy_row = crosstab.loc[hierarchy]
    top_2_leiden = hierarchy_row.nlargest(2)
    
    for leiden_cluster, count in top_2_leiden.items():
        if count > 0:  # Only include non-zero connections
            j = list(crosstab.columns).index(leiden_cluster)
            source_nodes.append(i)
            target_nodes.append(len(hierarchy_labels) + j)
            values.append(count)

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = labels,
        color = "blue"
    ),
    link = dict(
        source = source_nodes,
        target = target_nodes,
        value = values
    ))])

fig.update_layout(title_text="High Hierarchy to Leiden Cluster Distribution (Top 2 Connections)", 
                  font_size=10,
                  width=1000, 
                  height=2000)
fig.show()

In [ ]:
cluster = '10'

summary = adatas.obs.groupby('leiden')['high_hierarchy'].value_counts()
print("Top 5 high_hierarchy for cluster", cluster)
print(summary.loc[cluster].nlargest(5))
print()

summary = adatas.obs.groupby('leiden')['Original_annotation'].value_counts()
print("Top 5 Original_annotation for cluster", cluster)
print(summary.loc[cluster].nlargest(5))
print()

# Get the top high_hierarchy group for this cluster to find related alignment info
top_hierarchy = adatas.obs.groupby('leiden')['high_hierarchy'].value_counts().loc[cluster].index[0]
# Use the groups column directly instead of index filtering
matching_groups = alignment.relation[alignment.groups == top_hierarchy]
print("Related alignment groups:")
print(matching_groups)

In [ ]:
adatas.obs['Consensus_annotation_detailed'] = 'Unassigned'

categories = ['CD14 Mono', 'CD4 T Naive', 'B', 'Immature B', 'NK CD56 dim', 'CD8 T Memory',
               'CD8 T Naive', 'ErP', 'CD16 Mono', 'MAIT', 'cDC2', 'cDC1', 'NK CD56 bright',
               'pDC', 'CD4 CTL', 'Plasma', 'Unassigned', 'B Naive', 'B Memory', 'Stroma',
               'Gamma delta T', 'Progenitors', 'MEP','Treg', 'Doublet', 'Platelet', 'CD4 T Memory',
               'Mesenchymal','MkP', 'Late ErP', 'EoBaMaP', 'MPP/MEP', 'ILC', 'Macrophage']

adatas.obs['Consensus_annotation_detailed'] = pd.Categorical(adatas.obs['Consensus_annotation_detailed'], categories=categories)
adatas.obs.loc[adatas.obs['leiden'] == '0', 'Consensus_annotation_detailed'] = 'NK CD56 dim'
adatas.obs.loc[adatas.obs['leiden'] == '1', 'Consensus_annotation_detailed'] = 'CD14 Mono'
adatas.obs.loc[adatas.obs['leiden'] == '2', 'Consensus_annotation_detailed'] = 'Progenitors'
adatas.obs.loc[adatas.obs['leiden'] == '3', 'Consensus_annotation_detailed'] = 'CD4 T Naive'
adatas.obs.loc[adatas.obs['leiden'] == '4', 'Consensus_annotation_detailed'] = 'CD14 Mono'
#
adatas.obs.loc[adatas.obs['leiden'] == '5', 'Consensus_annotation_detailed'] = 'Progenitors'
adatas.obs.loc[adatas.obs['leiden'] == '6', 'Consensus_annotation_detailed'] = 'B Naive'
adatas.obs.loc[adatas.obs['leiden'] == '7', 'Consensus_annotation_detailed'] = 'CD8 T Naive'
adatas.obs.loc[adatas.obs['leiden'] == '8', 'Consensus_annotation_detailed'] = 'CD4 T Memory'
adatas.obs.loc[adatas.obs['leiden'] == '9', 'Consensus_annotation_detailed'] = 'Progenitors'
adatas.obs.loc[adatas.obs['leiden'] == '10', 'Consensus_annotation_detailed'] = 'Immature B'
adatas.obs.loc[adatas.obs['leiden'] == '11', 'Consensus_annotation_detailed'] = 'CD4 T Memory'
adatas.obs.loc[adatas.obs['leiden'] == '12', 'Consensus_annotation_detailed'] = 'CD14 Mono'
adatas.obs.loc[adatas.obs['leiden'] == '13', 'Consensus_annotation_detailed'] = 'CD8 T Memory'
adatas.obs.loc[adatas.obs['leiden'] == '14', 'Consensus_annotation_detailed'] = 'CD16 Mono'
adatas.obs.loc[adatas.obs['leiden'] == '15', 'Consensus_annotation_detailed'] = 'B Memory'
adatas.obs.loc[adatas.obs['leiden'] == '16', 'Consensus_annotation_detailed'] = 'CD8 T Memory'
adatas.obs.loc[adatas.obs['leiden'] == '17', 'Consensus_annotation_detailed'] = 'cDC2'
adatas.obs.loc[adatas.obs['leiden'] == '18', 'Consensus_annotation_detailed'] = 'pDC'
adatas.obs.loc[adatas.obs['leiden'] == '19', 'Consensus_annotation_detailed'] = 'Progenitors'
adatas.obs.loc[adatas.obs['leiden'] == '20', 'Consensus_annotation_detailed'] = 'MAIT'
adatas.obs.loc[adatas.obs['leiden'] == '21', 'Consensus_annotation_detailed'] = 'Progenitors'
adatas.obs.loc[adatas.obs['leiden'] == '22', 'Consensus_annotation_detailed'] = 'Gamma delta T'
adatas.obs.loc[adatas.obs['leiden'] == '23', 'Consensus_annotation_detailed'] = 'CD14 Mono'
adatas.obs.loc[adatas.obs['leiden'] == '24', 'Consensus_annotation_detailed'] = 'CD14 Mono'
adatas.obs.loc[adatas.obs['leiden'] == '25', 'Consensus_annotation_detailed'] = 'NK CD56 bright'
adatas.obs.loc[adatas.obs['leiden'] == '26', 'Consensus_annotation_detailed'] = 'Treg'
adatas.obs.loc[adatas.obs['leiden'] == '27', 'Consensus_annotation_detailed'] = 'Progenitors'
adatas.obs.loc[adatas.obs['leiden'] == '28', 'Consensus_annotation_detailed'] = 'CD8 T Memory'
adatas.obs.loc[adatas.obs['leiden'] == '29', 'Consensus_annotation_detailed'] = 'CD4 CTL'
adatas.obs.loc[adatas.obs['leiden'] == '30', 'Consensus_annotation_detailed'] = 'CD4 T Naive'
adatas.obs.loc[adatas.obs['leiden'] == '31', 'Consensus_annotation_detailed'] = 'CD4 T Naive'
adatas.obs.loc[adatas.obs['leiden'] == '32', 'Consensus_annotation_detailed'] = 'CD8 T Memory'
adatas.obs.loc[adatas.obs['leiden'] == '33', 'Consensus_annotation_detailed'] = 'Plasma'
adatas.obs.loc[adatas.obs['leiden'] == '34', 'Consensus_annotation_detailed'] = 'CD14 Mono'
adatas.obs.loc[adatas.obs['leiden'] == '35', 'Consensus_annotation_detailed'] = 'CD4 T Memory'
adatas.obs.loc[adatas.obs['leiden'] == '36', 'Consensus_annotation_detailed'] = 'B Memory'
adatas.obs.loc[adatas.obs['leiden'] == '37', 'Consensus_annotation_detailed'] = 'Progenitors'
adatas.obs.loc[adatas.obs['leiden'] == '38', 'Consensus_annotation_detailed'] = 'Stroma'
adatas.obs.loc[adatas.obs['leiden'] == '39', 'Consensus_annotation_detailed'] = 'CD16 Mono'
adatas.obs.loc[adatas.obs['leiden'] == '40', 'Consensus_annotation_detailed'] = 'CD4 T Memory'
adatas.obs.loc[adatas.obs['leiden'] == '41', 'Consensus_annotation_detailed'] = 'Progenitors'
adatas.obs.loc[adatas.obs['leiden'] == '42', 'Consensus_annotation_detailed'] = 'ILC'
adatas.obs.loc[adatas.obs['leiden'] == '43', 'Consensus_annotation_detailed'] = 'cDC1'
adatas.obs.loc[adatas.obs['leiden'] == '44', 'Consensus_annotation_detailed'] = 'Macrophage'
adatas.obs.loc[adatas.obs['leiden'] == '45', 'Consensus_annotation_detailed'] = 'NK CD56 dim'
adatas.obs.loc[adatas.obs['leiden'] == '46', 'Consensus_annotation_detailed'] = 'CD14 Mono'
adatas.obs.loc[adatas.obs['leiden'] == '47', 'Consensus_annotation_detailed'] = 'CD14 Mono'

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed_colors' in adatas.uns:
    del adatas.uns['Consensus_annotation_detailed_colors']

# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Draft consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_Consensus_annotation_detailed_draft_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Extract CD4 T Naive, CD4 T Memory, CD8 T Memory, MAIT, and Gamma delta T cells
t_cell_types = ['CD4 T Naive', 'CD4 T Memory', 'CD8 T Memory', 'CD8 T Naive', 'MAIT', 'Gamma delta T']
t_cell_mask = adatas.obs['Consensus_annotation_detailed'].isin(t_cell_types)
t_cell_subset = adatas[t_cell_mask].copy()

print(f"Number of T cells: {t_cell_subset.n_obs}")
print(f"Leiden clusters containing T cells: {t_cell_subset.obs['leiden'].unique()}")

# Check distribution of cell types
print("\nDistribution of T cell types:")
print(t_cell_subset.obs['Consensus_annotation_detailed'].value_counts())

# Perform subclustering on T cells
sc.pp.neighbors(t_cell_subset, use_rep="X_pcahm", n_neighbors=15, metric='cosine', random_state=42)
sc.tl.leiden(t_cell_subset, resolution=3, random_state=42, key_added='t_cell_subclusters')

# Create UMAP for the subset
sc.tl.umap(t_cell_subset, random_state=42, min_dist=0.3)

# Plot the subclusters
sc.pl.embedding(t_cell_subset, 
                color='t_cell_subclusters', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.set_title('T Cell Subclustering', fontsize=12, fontweight='bold', y=1.1)

plt.show()

# Plot original annotations
sc.pl.embedding(t_cell_subset, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.set_title('T Cell Original Annotations', fontsize=12, fontweight='bold', y=1.1)

plt.show()

# Check original annotations within each subcluster
print("\nOriginal annotations per subcluster:")
for cluster in sorted(t_cell_subset.obs['t_cell_subclusters'].unique()):
    cluster_cells = t_cell_subset.obs[t_cell_subset.obs['t_cell_subclusters'] == cluster]
    print(f"\nSubcluster {cluster}:")
    print(cluster_cells['Original_annotation'].value_counts().head())

# Find marker genes for subclusters
sc.tl.rank_genes_groups(t_cell_subset, 't_cell_subclusters', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(t_cell_subset, n_genes=5, sharey=False, ncols=3, fontsize=12)

plt.show()

In [ ]:
# First, add the new categories to both the subset and main adatas object
t_cell_subset.obs['t_cell_subclusters'] = t_cell_subset.obs['t_cell_subclusters'].cat.add_categories(['MAIT_updated', 'CD4_T_Memory_updated', 'CD4_T_Naive_updated', 'Gamma_delta_T_updated', 'CD8_T_Naive_updated', 'CD8_T_Memory_updated'])

# Add new categories to the main adatas categories if not already present
new_categories = ['MAIT', 'CD4 T Memory', 'CD4 T Naive', 'Gamma delta T', 'CD8 T Naive', 'CD8 T Memory']
for cat in new_categories:
    if cat not in adatas.obs['Consensus_annotation_detailed'].cat.categories:
        adatas.obs['Consensus_annotation_detailed'] = adatas.obs['Consensus_annotation_detailed'].cat.add_categories([cat])

# Assign subcluster 11 to Gamma delta T
t_cell_subset.obs.loc[t_cell_subset.obs['t_cell_subclusters'].isin(['11']), 't_cell_subclusters'] = 'Gamma_delta_T_updated'

# Assign subcluster 16 to MAIT
t_cell_subset.obs.loc[t_cell_subset.obs['t_cell_subclusters'].isin(['16']), 't_cell_subclusters'] = 'MAIT_updated'

# Assign subclusters 14, 30, 3, 13, 12, 21, 2, 36, 31, 27 to CD8 T Memory
t_cell_subset.obs.loc[t_cell_subset.obs['t_cell_subclusters'].isin(['14', '30', '3', '13', '12', '21', '2', '36', '31', '27', '22']), 't_cell_subclusters'] = 'CD8_T_Memory_updated'

# Assign subclusters 35, 0, 7, 19 to CD8 T Naive
t_cell_subset.obs.loc[t_cell_subset.obs['t_cell_subclusters'].isin(['35', '0', '7', '19']), 't_cell_subclusters'] = 'CD8_T_Naive_updated'

# Assign subclusters 25, 9, 23, 4, 10, 18, 34, 17, 24 to CD4 T Naive
t_cell_subset.obs.loc[t_cell_subset.obs['t_cell_subclusters'].isin(['25', '9', '23', '4', '10', '18', '34', '17', '24']), 't_cell_subclusters'] = 'CD4_T_Naive_updated'

# Assign remaining clusters to CD4 T Memory
remaining_clusters = t_cell_subset.obs['t_cell_subclusters'].unique()
remaining_clusters = [c for c in remaining_clusters if c not in ['MAIT_updated', 'CD4_T_Naive_updated', 'Gamma_delta_T_updated', 'CD8_T_Naive_updated', 'CD8_T_Memory_updated']]
t_cell_subset.obs.loc[t_cell_subset.obs['t_cell_subclusters'].isin(remaining_clusters), 't_cell_subclusters'] = 'CD4_T_Memory_updated'

# Update the main adatas object with the reassigned annotations
# MAIT cells
mait_reassigned_cells = t_cell_subset.obs[t_cell_subset.obs['t_cell_subclusters'] == 'MAIT_updated'].index
adatas.obs.loc[mait_reassigned_cells, 'Consensus_annotation_detailed'] = 'MAIT'

# CD4 T Memory cells
cd4_memory_reassigned_cells = t_cell_subset.obs[t_cell_subset.obs['t_cell_subclusters'] == 'CD4_T_Memory_updated'].index
adatas.obs.loc[cd4_memory_reassigned_cells, 'Consensus_annotation_detailed'] = 'CD4 T Memory'

# CD4 T Naive cells
cd4_naive_reassigned_cells = t_cell_subset.obs[t_cell_subset.obs['t_cell_subclusters'] == 'CD4_T_Naive_updated'].index
adatas.obs.loc[cd4_naive_reassigned_cells, 'Consensus_annotation_detailed'] = 'CD4 T Naive'

# Gamma delta T cells
gamma_delta_reassigned_cells = t_cell_subset.obs[t_cell_subset.obs['t_cell_subclusters'] == 'Gamma_delta_T_updated'].index
adatas.obs.loc[gamma_delta_reassigned_cells, 'Consensus_annotation_detailed'] = 'Gamma delta T'

# CD8 T Naive cells
cd8_naive_reassigned_cells = t_cell_subset.obs[t_cell_subset.obs['t_cell_subclusters'] == 'CD8_T_Naive_updated'].index
adatas.obs.loc[cd8_naive_reassigned_cells, 'Consensus_annotation_detailed'] = 'CD8 T Naive'

# CD8 T Memory cells
cd8_memory_reassigned_cells = t_cell_subset.obs[t_cell_subset.obs['t_cell_subclusters'] == 'CD8_T_Memory_updated'].index
adatas.obs.loc[cd8_memory_reassigned_cells, 'Consensus_annotation_detailed'] = 'CD8 T Memory'

# Print summary of reassignments
print("T cell reassignment summary:")
print(f"MAIT: {len(mait_reassigned_cells)} cells")
print(f"CD4 T Memory: {len(cd4_memory_reassigned_cells)} cells")
print(f"CD4 T Naive: {len(cd4_naive_reassigned_cells)} cells")
print(f"Gamma delta T: {len(gamma_delta_reassigned_cells)} cells")
print(f"CD8 T Naive: {len(cd8_naive_reassigned_cells)} cells")
print(f"CD8 T Memory: {len(cd8_memory_reassigned_cells)} cells")

# Plot updated annotations
sc.pl.embedding(t_cell_subset, 
                color='t_cell_subclusters', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.set_title('T Cell Updated Annotations', fontsize=12, fontweight='bold', y=1.1)

plt.show()

In [ ]:
# Extract Immature B cells
immature_b_mask = adatas.obs['Consensus_annotation_detailed'] == 'Immature B'
immature_b_subset = adatas[immature_b_mask].copy()

print(f"Number of Immature B cells: {immature_b_subset.n_obs}")
print(f"Leiden clusters containing Immature B: {immature_b_subset.obs['leiden'].unique()}")

# Perform subclustering on Immature B cells
sc.pp.neighbors(immature_b_subset, use_rep="X_pcahm", n_neighbors=15, metric='cosine', random_state=42)
sc.tl.leiden(immature_b_subset, resolution=0.5, random_state=42, key_added='immature_b_subclusters')

# Create UMAP for the subset
sc.tl.umap(immature_b_subset, random_state=42, min_dist=0.3)

# Plot the subclusters
sc.pl.embedding(immature_b_subset, 
                color='immature_b_subclusters', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.set_title('Immature B Subclustering', fontsize=12, fontweight='bold', y=1.1)

plt.show()

# Check original annotations within each subcluster
print("\nOriginal annotations per subcluster:")
for cluster in immature_b_subset.obs['immature_b_subclusters'].unique():
    cluster_cells = immature_b_subset.obs[immature_b_subset.obs['immature_b_subclusters'] == cluster]
    print(f"\nSubcluster {cluster}:")
    print(cluster_cells['Original_annotation'].value_counts().head())

# Find marker genes for subclusters
sc.tl.rank_genes_groups(immature_b_subset, 'immature_b_subclusters', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(immature_b_subset, n_genes=5, sharey=False, ncols=3, fontsize=12)

plt.show()

In [ ]:
# Plot the subclusters
sc.pl.embedding(immature_b_subset, 
                color='CellTypes', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

plt.show()

In [ ]:
# First, add the new categories to both the subset and main adatas object
immature_b_subset.obs['immature_b_subclusters'] = immature_b_subset.obs['immature_b_subclusters'].cat.add_categories(['Pre-B'])

# Add Pre-B to the main adatas categories if not already present
if 'Pre-B' not in adatas.obs['Consensus_annotation_detailed'].cat.categories:
    adatas.obs['Consensus_annotation_detailed'] = adatas.obs['Consensus_annotation_detailed'].cat.add_categories(['Pre-B'])

# Now reassign subcluster 2 to Pre-B
immature_b_subset.obs.loc[immature_b_subset.obs['immature_b_subclusters'].isin(['2']), 'immature_b_subclusters'] = 'Pre-B'

# Update the main adatas object with the reassigned annotations
# Update cells from subcluster 2 that are now labeled as Pre-B
immature_b_reassigned_cells = immature_b_subset.obs[immature_b_subset.obs['immature_b_subclusters'] == 'Pre-B'].index
adatas.obs.loc[immature_b_reassigned_cells, 'Consensus_annotation_detailed'] = 'Pre-B'

# Update remaining Immature B cells (those not reassigned)
immature_b_remaining_cells = immature_b_subset.obs[immature_b_subset.obs['immature_b_subclusters'] == 'Immature B'].index
adatas.obs.loc[immature_b_remaining_cells, 'Consensus_annotation_detailed'] = 'Immature B'

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Draft consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_Consensus_annotation_detailed_draft_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
adatas.obs['Consensus_annotation_broad'] = 'Mature'

categories = ['Mature', 'Immature']

adatas.obs['Consensus_annotation_broad'] = pd.Categorical(adatas.obs['Consensus_annotation_broad'], categories=categories)
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'Progenitors', 'Consensus_annotation_broad'] = 'Immature'

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='Consensus_annotation_broad', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Draft consensus broad annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_consensus_annotation_broad_draft_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
Zhang_dataset.obs['Consensus_annotation_detailed']=adatas.obs.loc[Zhang_dataset.obs_names, 'Consensus_annotation_detailed'].values
Zhang_dataset.obs['Consensus_annotation_broad']=adatas.obs.loc[Zhang_dataset.obs_names, 'Consensus_annotation_broad'].values

Hao_dataset.obs['Consensus_annotation_detailed']=adatas.obs.loc[Hao_dataset.obs_names, 'Consensus_annotation_detailed'].values
Hao_dataset.obs['Consensus_annotation_broad']=adatas.obs.loc[Hao_dataset.obs_names, 'Consensus_annotation_broad'].values

Triana_dataset.obs['Consensus_annotation_detailed']=adatas.obs.loc[Triana_dataset.obs_names, 'Consensus_annotation_detailed'].values
Triana_dataset.obs['Consensus_annotation_broad']=adatas.obs.loc[Triana_dataset.obs_names, 'Consensus_annotation_broad'].values

Luecken_dataset.obs['Consensus_annotation_detailed']=adatas.obs.loc[Luecken_dataset.obs_names, 'Consensus_annotation_detailed'].values
Luecken_dataset.obs['Consensus_annotation_broad']=adatas.obs.loc[Luecken_dataset.obs_names, 'Consensus_annotation_broad'].values

### HSPC

In [ ]:
original_labels = pd.Categorical(np.concatenate((Zhang_dataset.obs['Level 3 Multimodal'].values,
                                                 Triana_dataset.obs['CellTypes'].values, 
                                                 Luecken_dataset.obs['cell_type'].values)))

In [ ]:
adatas_HSPC = {"Zhang": Zhang_dataset, 
               "Triana": Triana_dataset, 
               "Luecken": Luecken_dataset}

adatas_HSPC = anndata.concat(adatas_HSPC, 
                             label="dataset_name", 
                             join="outer")

In [ ]:
common = SCUtils.Intersect_lists(Zhang_dataset.var_names, 
                                 Triana_dataset.var_names, 
                                 Luecken_dataset.var_names)

In [ ]:
from venny4py.venny4py import *

# Create the Venn diagram with custom colors
sets = {'Zhang': set(list(Zhang_dataset.var_names)),
        'Triana': set(list(Triana_dataset.var_names)),
        'Luecken': set(list(Luecken_dataset.var_names))}

# Define custom colors for each dataset
colors = ['#1F77B4',  # Zhang - blue
          '#2EA02E',  # Triana - green
          '#D62828']  # Luecken - red

venny4py(sets=sets, out=figures_path, ext='png', colors=colors)

# Display the plot
plt.show()

# Specify the current file name and the new file name
current_file_name = figures_path + "/Venn_3.png"
new_file_name = figures_path + "/Shared_features_across_hspcs_containing_datasets.png"

# Rename the file
os.rename(current_file_name, new_file_name)

# Specify the current file name and the new file name
current_file_name = figures_path + "/Intersections_3.txt"
new_file_name = figures_path + "/Shared_features_across_hspcs_containing_datasets_list.txt"

# Rename the file
os.rename(current_file_name, new_file_name)

In [ ]:
adatas_HSPC = adatas_HSPC[:, common]

In [ ]:
adatas_HSPC.obs['Original_annotation'] = original_labels

In [ ]:
immature_obs_names = adatas.obs_names[(adatas.obs['Consensus_annotation_broad'] == 'Immature') & (adatas.obs['dataset_name'] != 'Hao')]
obs_to_keep = np.in1d(adatas_HSPC.obs_names, immature_obs_names)

In [ ]:
adatas_HSPC = adatas_HSPC[obs_to_keep,:]

In [ ]:
adatas_HSPC.X = SCUtils.Protein_normalization(adatas_HSPC.X)
sc.pp.regress_out(adatas_HSPC, keys=['Chemistry'])

In [ ]:
adatas_HSPC

In [ ]:
random.seed(42)
np.random.seed(42)

ho = hm.run_harmony(adatas_HSPC.X, adatas_HSPC.obs, ["Batch"], 
                    max_iter_harmony = 30, random_state = 42)

In [ ]:
pc_std = np.std(ho.Z_corr, axis=1).tolist()

In [ ]:
sns.scatterplot(x=range(0, len(pc_std)), y=sorted(pc_std, reverse=True))

In [ ]:
adatas_HSPC.obsm["X_pcahm"] = ho.Z_corr.transpose()

In [ ]:
sc.pp.neighbors(adatas_HSPC, use_rep="X_pcahm", n_neighbors=30, metric='cosine', random_state = 42, n_pcs=40)

adatas_HSPC.obsp["connectivities"] = np.round(adatas_HSPC.obsp["connectivities"], decimals=1)
adatas_HSPC.obsp["distances"] = np.round(adatas_HSPC.obsp["distances"], decimals=1)

In [ ]:
sc.tl.umap(adatas_HSPC, random_state = 42,  min_dist=0.2)

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas_HSPC, 
                color='dataset_name', 
                basis='X_umap', 
                legend_loc='right margin', 
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(7, 8)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.05)

# Place the legend below the plot
legend = ax.legend(loc='upper center', 
                   bbox_to_anchor=(0.1, 1),
                   prop={'size': 4.8},
                   ncol=1)

# Reduce the size of the dots in the legend
for handle in legend.legend_handles:
    handle._sizes = [10]

# Adjust the layout to make room for the legend
plt.subplots_adjust(bottom=0.3)

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_hspcs_datasets_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Create subplot figure split by dataset_name
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

# Get unique datasets
datasets = adatas_HSPC.obs['dataset_name'].unique()

# Define colors for each dataset
dataset_colors = {
    'Zhang': '#1F77B4',   # blue
    'Triana': '#2EA02E',  # green  
    'Luecken': '#D62828'  # red
}

for i, dataset in enumerate(datasets):
    # Filter data for current dataset
    dataset_mask = adatas_HSPC.obs['dataset_name'] == dataset
    dataset_data = adatas_HSPC[dataset_mask]
    
    # Plot UMAP for current dataset with specific color
    sc.pl.embedding(dataset_data, 
                    color='dataset_name', 
                    basis='X_umap', 
                    legend_loc='none',
                    add_outline=False,
                    frameon=False,
                    show=False,
                    ax=axes[i],
                    palette=[dataset_colors[dataset]])
    
    # Set axis labels and title for each subplot
    axes[i].set_xlabel('UMAP 1', fontsize=12)
    axes[i].set_ylabel('UMAP 2', fontsize=12)
    axes[i].set_title(f'{dataset} dataset', fontsize=12, fontweight='bold')

# Remove empty subplot if odd number of datasets
if len(datasets) < 4:
    fig.delaxes(axes[3])

# Adjust layout
plt.tight_layout()

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_HSPC_containing_datasets_split_by_dataset.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
sc.tl.leiden(adatas_HSPC, resolution=3, random_state = 42, n_iterations=10)

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas_HSPC, 
                color='leiden', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Cluster annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_hspcs_leiden_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas_HSPC, 
                color='cell_type', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Cluster annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
alignment = cellhint.harmonize(adatas_HSPC, 'dataset_name', 'Original_annotation', 
                               use_rep='X_pcahm', metric='cosine')

In [ ]:
cellhint.treeplot(alignment, save=figures_path + "/Merged_datasets_hspcs_cellhint.png")

In [ ]:
adatas_HSPC.obs[['low_hierarchy', 'high_hierarchy']] = alignment.reannotation.loc[adatas_HSPC.obs_names, ['reannotation', 'group']]

In [ ]:
adatas_HSPC.obs['low_hierarchy'] = pd.Categorical(adatas_HSPC.obs['low_hierarchy'])
adatas_HSPC.obs['high_hierarchy'] = pd.Categorical(adatas_HSPC.obs['high_hierarchy'])

In [ ]:
cluster = '7'

summary = adatas_HSPC.obs.groupby('leiden')['high_hierarchy'].value_counts()
print("Top 5 high_hierarchy for cluster", cluster)
print(summary.loc[cluster].nlargest(5))
print()

summary = adatas_HSPC.obs.groupby('leiden')['Original_annotation'].value_counts()
print("Top 5 Original_annotation for cluster", cluster)
print(summary.loc[cluster].nlargest(5))
print()

# Get the top high_hierarchy group for this cluster to find related alignment info
top_hierarchy = adatas_HSPC.obs.groupby('leiden')['high_hierarchy'].value_counts().loc[cluster].index[0]
# Use the groups column directly instead of index filtering
matching_groups = alignment.relation[alignment.groups == top_hierarchy]
print("Related alignment groups:")
print(matching_groups)

In [ ]:
adatas_HSPC.obs['Consensus_annotation_detailed'] = 'Unassigned'

categories = ['MEP','ErP', 'EoBaMaP', 'Myeloid progenitor', 'Erythroblast', 'Neutrophil progenitor', 'pDC', 'NK progenitor',
              'HSC', 'MPP','GMP','Lymphoid progenitor','MkP','LMPP', 'Immature B', 'CD14 Mono', 'Progenitors', 'Pre-Pro-B', 'Pro-B']

adatas_HSPC.obs['Consensus_annotation_detailed'] = pd.Categorical(adatas_HSPC.obs['Consensus_annotation_detailed'], categories=categories)
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '0', 'Consensus_annotation_detailed'] = 'Myeloid progenitor'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '1', 'Consensus_annotation_detailed'] = 'MPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '2', 'Consensus_annotation_detailed'] = 'HSC'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '3', 'Consensus_annotation_detailed'] = 'GMP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '4', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '5', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '6', 'Consensus_annotation_detailed'] = 'LMPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '7', 'Consensus_annotation_detailed'] = 'MPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '8', 'Consensus_annotation_detailed'] = 'MEP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '9', 'Consensus_annotation_detailed'] = 'LMPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '10', 'Consensus_annotation_detailed'] = 'Erythroblast'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '11', 'Consensus_annotation_detailed'] = 'LMPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '12', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '13', 'Consensus_annotation_detailed'] = 'pDC'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '14', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '15', 'Consensus_annotation_detailed'] = 'MkP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '16', 'Consensus_annotation_detailed'] = 'Pre-Pro-B'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '17', 'Consensus_annotation_detailed'] = 'LMPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '18', 'Consensus_annotation_detailed'] = 'LMPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '19', 'Consensus_annotation_detailed'] = 'GMP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '20', 'Consensus_annotation_detailed'] = 'Pro-B'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '21', 'Consensus_annotation_detailed'] = 'EoBaMaP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '22', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '23', 'Consensus_annotation_detailed'] = 'LMPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '24', 'Consensus_annotation_detailed'] = 'MPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '25', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '26', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '27', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '28', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '29', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '30', 'Consensus_annotation_detailed'] = 'MEP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '31', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '32', 'Consensus_annotation_detailed'] = 'ErP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '33', 'Consensus_annotation_detailed'] = 'LMPP'
adatas_HSPC.obs.loc[adatas_HSPC.obs['leiden'] == '34', 'Consensus_annotation_detailed'] = 'LMPP'

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed_colors' in adatas_HSPC.uns:
    del adatas_HSPC.uns['Consensus_annotation_detailed_colors']

# Plot UMAP with color
sc.pl.embedding(adatas_HSPC, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Draft consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_hspcs_Consensus_annotation_detailed_draft_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

## Joint annotation

In [ ]:
# Get categories from both datasets
adatas_categories = list(adatas.obs['Consensus_annotation_detailed'].cat.categories)
adatas_hspc_categories = list(adatas_HSPC.obs['Consensus_annotation_detailed'].cat.categories)

# Merge categories and remove duplicates while preserving order
merged_categories = []
for cat in adatas_categories + adatas_hspc_categories:
    if cat not in merged_categories:
        merged_categories.append(cat)

# Set the merged categories for adatas
adatas.obs['Consensus_annotation_detailed'] = pd.Categorical(
    adatas.obs['Consensus_annotation_detailed'], 
    categories=merged_categories
)

In [ ]:
adatas.obs.loc[adatas_HSPC.obs.index, 'Consensus_annotation_detailed'] = adatas_HSPC.obs['Consensus_annotation_detailed'].astype(str)

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_Consensus_annotation_detailed_final_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
adatas.obs['Consensus_annotation_broad'] = 'Mature'

categories = ['Mature', 'Immature']

adatas.obs['Consensus_annotation_broad'] = pd.Categorical(adatas.obs['Consensus_annotation_broad'], categories=categories)
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'HSC_MPP', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'ErP', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'MEP', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'GMP', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'Myeloid progenitor', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'Neutrophil progenitor', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'NK progenitor', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'pDC progenitor', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'Pro-B', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'Pre-Pro-B', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'LMPP', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'MkP', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'EoBaMaP', 'Consensus_annotation_broad'] = 'Immature'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'Progenitors', 'Consensus_annotation_broad'] = 'Immature'

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='Consensus_annotation_broad', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus broad annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_consensus_annotation_broad_final_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
adatas.obs['Consensus_annotation_simplified'] = ''

categories = ['HSPC', 'Monocyte', 'Myeloid progenitor', 'NK', 'CD4 T', 'CD8 T', 'B', 'Erythroid', 'Doublet', 'Stroma', 'ILC', 'cDC', 'pDC']

adatas.obs['Consensus_annotation_simplified'] = pd.Categorical(adatas.obs['Consensus_annotation_simplified'], categories=categories)
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'].isin(['HSC_MPP', 'MEP','GMP','Pre-Pro-B','Pro-B', 'MkP','EoBaMaP', 'NK progenitor', 'pDC progenitor', 'Neutrophil progenitor', 'Progenitors']), 'Consensus_annotation_simplified'] = 'HSPC'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'].isin(['CD14 Mono', 'CD16 Mono', 'Macrophage']), 'Consensus_annotation_simplified'] = 'Monocyte'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'].isin(['Myeloid progenitor']), 'Consensus_annotation_simplified'] = 'Myeloid progenitor'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'].isin(['NK CD56 dim', 'NK CD56 bright']), 'Consensus_annotation_simplified'] = 'NK'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'].isin(['CD4 T Naive', 'CD4 T Memory', 'Treg', 'CD4 CTL']), 'Consensus_annotation_simplified'] = 'CD4 T'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'].isin(['CD8 T Naive', 'CD8 T Memory', 'MAIT', 'Gamma delta T']), 'Consensus_annotation_simplified'] = 'CD8 T'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'].isin(['B Naive', 'B Memory', 'Plasma', 'Immature B', 'Pre-B']), 'Consensus_annotation_simplified'] = 'B'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'].isin(['Erythroblast', 'Platelet', 'ErP']), 'Consensus_annotation_simplified'] = 'Erythroid'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'Doublet', 'Consensus_annotation_simplified'] = 'Doublet'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'Stroma', 'Consensus_annotation_simplified'] = 'Stroma'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'ILC', 'Consensus_annotation_simplified'] = 'ILC'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'].isin(['cDC2', 'cDC1']), 'Consensus_annotation_simplified'] = 'cDC'
adatas.obs.loc[adatas.obs['Consensus_annotation_detailed'] == 'pDC', 'Consensus_annotation_simplified'] = 'pDC'

In [ ]:
# Plot UMAP with color
sc.pl.embedding(adatas, 
                color='Consensus_annotation_simplified', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus simplified annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Merged_datasets_consensus_annotation_simplified_final_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
adatas.write_h5ad(data_path + "/References/Merged_references.h5ad")

## Re-labelling

In [ ]:
Zhang_dataset.obs['Consensus_annotation_broad']=adatas.obs.loc[Zhang_dataset.obs_names, 'Consensus_annotation_broad'].values
Zhang_dataset.obs['Consensus_annotation_simplified']=adatas.obs.loc[Zhang_dataset.obs_names, 'Consensus_annotation_simplified'].values
Zhang_dataset.obs['Consensus_annotation_detailed']=adatas.obs.loc[Zhang_dataset.obs_names, 'Consensus_annotation_detailed'].values

Hao_dataset.obs['Consensus_annotation_broad']=adatas.obs.loc[Hao_dataset.obs_names, 'Consensus_annotation_broad'].values
Hao_dataset.obs['Consensus_annotation_simplified']=adatas.obs.loc[Hao_dataset.obs_names, 'Consensus_annotation_simplified'].values
Hao_dataset.obs['Consensus_annotation_detailed']=adatas.obs.loc[Hao_dataset.obs_names, 'Consensus_annotation_detailed'].values

Triana_dataset.obs['Consensus_annotation_broad']=adatas.obs.loc[Triana_dataset.obs_names, 'Consensus_annotation_broad'].values
Triana_dataset.obs['Consensus_annotation_simplified']=adatas.obs.loc[Triana_dataset.obs_names, 'Consensus_annotation_simplified'].values
Triana_dataset.obs['Consensus_annotation_detailed']=adatas.obs.loc[Triana_dataset.obs_names, 'Consensus_annotation_detailed'].values

Luecken_dataset.obs['Consensus_annotation_broad']=adatas.obs.loc[Luecken_dataset.obs_names, 'Consensus_annotation_broad'].values
Luecken_dataset.obs['Consensus_annotation_simplified']=adatas.obs.loc[Luecken_dataset.obs_names, 'Consensus_annotation_simplified'].values
Luecken_dataset.obs['Consensus_annotation_detailed']=adatas.obs.loc[Luecken_dataset.obs_names, 'Consensus_annotation_detailed'].values

### Zhang dataset

In [ ]:
# Remove unused categories
Zhang_dataset.obs['Consensus_annotation_simplified'] = Zhang_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Zhang_dataset.obs['Consensus_annotation_detailed'] = Zhang_dataset.obs['Consensus_annotation_detailed'].cat.remove_unused_categories()

In [ ]:
counts = Zhang_dataset.obs['Consensus_annotation_detailed'].value_counts()

In [ ]:
print(counts)

In [ ]:
filtered_categories = counts[counts >= 10].index
Zhang_dataset = Zhang_dataset[Zhang_dataset.obs[Zhang_dataset.obs['Consensus_annotation_detailed'].isin(filtered_categories)].index, :]

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Zhang_dataset, 
                color='Level 3 Multimodal', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=2,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Zhang X. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation - smoothed', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed_colors' in Zhang_dataset.uns:
    del Zhang_dataset.uns['Consensus_annotation_detailed_colors']

# Plot UMAP with color
sc.pl.embedding(Zhang_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Zhang X. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation - smoothed', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
# Get current categories and safely add new ones
current_categories = list(Zhang_dataset.obs['Consensus_annotation_simplified'].cat.categories)

# Add new categories only if they don't already exist
new_cats_to_add = []
for cat in ['Erythroid', 'Myeloid']:  # Added 'Myeloid' here
    if cat not in current_categories:
        new_cats_to_add.append(cat)

if new_cats_to_add:
    all_categories = current_categories + new_cats_to_add
    Zhang_dataset.obs['Consensus_annotation_simplified'] = Zhang_dataset.obs['Consensus_annotation_simplified'].cat.set_categories(all_categories)

# Get current categories and safely add new ones for detailed annotation
current_categories = list(Zhang_dataset.obs['Consensus_annotation_detailed'].cat.categories)

# Add new categories only if they don't already exist
new_cats_to_add = []
for cat in ['Erythroblast', 'Transitional B', 'MAIT', 'Gamma delta T', 'cDC2', 'MPP', 'Platelet', 'MkP', 'EoBaMaP', 'Myeloid progenitor', 'Neutrophil progenitor', 'pDC progenitor', 'Progenitors', 'Myeloid']:
    if cat not in current_categories:
        new_cats_to_add.append(cat)

if new_cats_to_add:
    all_categories = current_categories + new_cats_to_add
    Zhang_dataset.obs['Consensus_annotation_detailed'] = Zhang_dataset.obs['Consensus_annotation_detailed'].cat.set_categories(all_categories)

# Now proceed with the assignments

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MEP-1', 'MEP-2']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MEP-1', 'MEP-2']), 'Consensus_annotation_simplified'] = 'HSPC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MEP-1', 'MEP-2']), 'Consensus_annotation_detailed'] = 'MEP'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Intermediate Mono-1','Intermediate Mono-2','Intermediate Mono-3','Classical-Mono']), 'Consensus_annotation_broad'] = 'Mature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Intermediate Mono-1','Intermediate Mono-2','Intermediate Mono-3','Classical-Mono']), 'Consensus_annotation_simplified'] = 'Monocyte'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Intermediate Mono-1','Intermediate Mono-2','Intermediate Mono-3','Classical-Mono']), 'Consensus_annotation_detailed'] = 'CD14 Mono'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['BMCP-1','BMCP-2']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['BMCP-1','BMCP-2']), 'Consensus_annotation_simplified'] = 'HSPC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['BMCP-1','BMCP-2']), 'Consensus_annotation_detailed'] = 'EoBaMaP'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['pre-DC-2','pre-DC-1','cDC1']), 'Consensus_annotation_broad'] = 'Mature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['pre-DC-2','pre-DC-1','cDC1']), 'Consensus_annotation_simplified'] = 'cDC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['pre-DC-2','pre-DC-1','cDC1']), 'Consensus_annotation_detailed'] = 'cDC1'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['pre-DC-3','cDC2-1', 'cDC2-2', 'ASDC']), 'Consensus_annotation_broad'] = 'Mature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['pre-DC-3','cDC2-1', 'cDC2-2', 'ASDC']), 'Consensus_annotation_simplified'] = 'cDC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['pre-DC-3','cDC2-1', 'cDC2-2', 'ASDC']), 'Consensus_annotation_detailed'] = 'cDC2'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MEP-Eryth-1','MEP-Eryth-2','ERP-1','ERP-2','ERP-3','ERP-4','ERP-5','ERP-6','ERP-7','ERP-8']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MEP-Eryth-1','MEP-Eryth-2','ERP-1','ERP-2','ERP-3','ERP-4','ERP-5','ERP-6','ERP-7','ERP-8']), 'Consensus_annotation_simplified'] = 'Erythroid'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MEP-Eryth-1','MEP-Eryth-2','ERP-1','ERP-2','ERP-3','ERP-4','ERP-5','ERP-6','ERP-7','ERP-8']), 'Consensus_annotation_detailed'] = 'ErP'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Erythroblast-1','Erythroblast-2','Erythroblast-3']), 'Consensus_annotation_broad'] = 'Mature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Erythroblast-1','Erythroblast-2','Erythroblast-3']), 'Consensus_annotation_simplified'] = 'Erythroid'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Erythroblast-1','Erythroblast-2','Erythroblast-3']), 'Consensus_annotation_detailed'] = 'Erythroblast'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['LMPP-1-cycling', 'LMPP-1']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['LMPP-1-cycling', 'LMPP-1']), 'Consensus_annotation_simplified'] = 'HSPC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['LMPP-1-cycling', 'LMPP-1']), 'Consensus_annotation_detailed'] = 'MPP'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['CLP']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['CLP']), 'Consensus_annotation_simplified'] = 'HSPC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['CLP']), 'Consensus_annotation_detailed'] = 'Pre-Pro-B'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['T CD8 Naive']), 'Consensus_annotation_broad'] = 'Mature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['T CD8 Naive']), 'Consensus_annotation_simplified'] = 'CD8 T'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['T CD8 Naive']), 'Consensus_annotation_detailed'] = 'CD8 T Naive'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Myeloid intermediate 1','Myeloid intermediate 2','Myeloid intermediate 3','Mono-1','Mono-2','cMOP']), 'Consensus_annotation_broad'] = 'Mature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Myeloid intermediate 1','Myeloid intermediate 2','Myeloid intermediate 3','Mono-1','Mono-2','cMOP']), 'Consensus_annotation_simplified'] = 'Myeloid'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Myeloid intermediate 1','Myeloid intermediate 2','Myeloid intermediate 3','Mono-1','Mono-2','cMOP']), 'Consensus_annotation_detailed'] = 'Myeloid progenitor'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['preNeu','immNeu-1','immNeu-2']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['preNeu','immNeu-1','immNeu-2']), 'Consensus_annotation_simplified'] = 'HSPC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['preNeu','immNeu-1','immNeu-2']), 'Consensus_annotation_detailed'] = 'GMP'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MPP-MEP']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MPP-MEP']), 'Consensus_annotation_simplified'] = 'HSPC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MPP-MEP']), 'Consensus_annotation_detailed'] = 'MPP'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['HSC-1','HSC-2', 'MPP-1', 'MPP-2']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['HSC-1','HSC-2', 'MPP-1','MPP-2']), 'Consensus_annotation_simplified'] = 'HSPC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['HSC-1','HSC-2', 'MPP-1','MPP-2']), 'Consensus_annotation_detailed'] = 'HSC'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MultiLin-GMP-1','MultiLin-GMP-2','MultiLin-GMP-3','Multilin-1','Multilin-2','Multilin-3','LMPP-2','MDP-1','MDP-2']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MultiLin-GMP-1','MultiLin-GMP-2','MultiLin-GMP-3','Multilin-1','Multilin-2','Multilin-3','LMPP-2','MDP-1','MDP-2']), 'Consensus_annotation_simplified'] = 'HSPC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['MultiLin-GMP-1','MultiLin-GMP-2','MultiLin-GMP-3','Multilin-1','Multilin-2','Multilin-3','LMPP-2','MDP-1','MDP-2']), 'Consensus_annotation_detailed'] = 'LMPP'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Pro-B-Early-cycling', 'Pro-B-Early', 'Pro-B-cycling-1','Pro-B-cycling-2', 'Pro-B-2', 'Pro-B-3', 'Pro-B-1']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Pro-B-Early-cycling', 'Pro-B-Early', 'Pro-B-cycling-1','Pro-B-cycling-2', 'Pro-B-2', 'Pro-B-3', 'Pro-B-1']), 'Consensus_annotation_simplified'] = 'B'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Pro-B-Early-cycling', 'Pro-B-Early', 'Pro-B-cycling-1','Pro-B-cycling-2', 'Pro-B-2', 'Pro-B-3', 'Pro-B-1']), 'Consensus_annotation_detailed'] = 'Pro-B'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Transitional-B-2']), 'Consensus_annotation_broad'] = 'Immature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Transitional-B-2']), 'Consensus_annotation_simplified'] = 'B'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Transitional-B-2']), 'Consensus_annotation_detailed'] = 'Pre-B'

Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Transitional-B-1', 'pre-B']), 'Consensus_annotation_broad'] = 'Mature'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Transitional-B-1', 'pre-B']), 'Consensus_annotation_simplified'] = 'B'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Level 3 Multimodal'].isin(['Transitional-B-1', 'pre-B']), 'Consensus_annotation_detailed'] = 'Immature B'


In [ ]:
# Remove unused categories
Zhang_dataset.obs['Consensus_annotation_simplified'] = Zhang_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Zhang_dataset.obs['Consensus_annotation_detailed'] = Zhang_dataset.obs['Consensus_annotation_detailed'].cat.remove_unused_categories()

Smoothing labels

In [ ]:
counts = Zhang_dataset.obs['Consensus_annotation_detailed'].value_counts()
print(counts)

In [ ]:
filtered_categories = counts[counts >= 10].index
Zhang_dataset = Zhang_dataset[Zhang_dataset.obs[Zhang_dataset.obs['Consensus_annotation_detailed'].isin(filtered_categories)].index, :]

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Calculate silhouette scores for current annotations
print("Calculating silhouette scores for Zhang dataset...")

# Use the UMAP representation for silhouette analysis
X_embed = Zhang_dataset.obsm['X_umap']
labels = Zhang_dataset.obs['Consensus_annotation_detailed'].astype('category').cat.codes

# Calculate silhouette scores
silhouette_avg = silhouette_score(X_embed, labels)
sample_silhouette_values = silhouette_samples(X_embed, labels)

print(f"Average silhouette score: {silhouette_avg:.3f}")

# Add silhouette scores to the dataset
Zhang_dataset.obs['silhouette_score'] = sample_silhouette_values

# Identify cells with negative silhouette scores
negative_silhouette_mask = sample_silhouette_values < 0
print(f"Number of cells with negative silhouette scores: {negative_silhouette_mask.sum()}")
print(f"Percentage of cells with negative silhouette scores: {negative_silhouette_mask.sum()/len(sample_silhouette_values)*100:.2f}%")

# Show distribution of silhouette scores by cell type
silhouette_by_type = Zhang_dataset.obs.groupby('Consensus_annotation_detailed')['silhouette_score'].agg(['mean', 'std', 'min', 'max', 'count'])
print("\nSilhouette scores by cell type:")
print(silhouette_by_type.sort_values('mean'))

# Initialize refined annotations (start with original smoothed annotations)
Zhang_dataset.obs['Consensus_annotation_detailed_refined'] = Zhang_dataset.obs['Consensus_annotation_detailed'].copy()

# Perform silhouette-based reassignment
print("\n=== PERFORMING SILHOUETTE-BASED REASSIGNMENT ===")

# Identify cells with very poor silhouette scores (< -0.1)
very_poor_silhouette = Zhang_dataset.obs['silhouette_score'] < -0.1

if very_poor_silhouette.sum() > 0:
    print(f"Found {very_poor_silhouette.sum()} cells with very poor silhouette scores (< -0.1)")
    
    # Fit nearest neighbors
    nn = NearestNeighbors(n_neighbors=30, metric='euclidean')
    nn.fit(X_embed)
    
    # Get indices of poorly assigned cells
    poor_indices = np.where(very_poor_silhouette)[0]
    
    reassignments_made = 0
    
    for idx in poor_indices:
        # Find neighbors for this cell
        distances, neighbor_indices = nn.kneighbors([X_embed[idx]])
        neighbor_indices = neighbor_indices[0][1:]  # Exclude the cell itself
        
        # Get annotations of neighbors
        neighbor_annotations = Zhang_dataset.obs['Consensus_annotation_detailed'].iloc[neighbor_indices]

        # Find most common annotation among neighbors
        most_common = neighbor_annotations.mode()
        
        if len(most_common) > 0:
            new_annotation = most_common.iloc[0]
            current_annotation = Zhang_dataset.obs['Consensus_annotation_detailed'].iloc[idx]
            
            # Only reassign if the most common neighbor annotation is different
            if new_annotation != current_annotation:
                # Check if at least 40% of neighbors have this annotation
                fraction = (neighbor_annotations == new_annotation).sum() / len(neighbor_annotations)
                
                if fraction >= 0.4:
                    Zhang_dataset.obs.loc[Zhang_dataset.obs.index[idx], 'Consensus_annotation_detailed_refined'] = new_annotation
                    reassignments_made += 1
    
    print(f"Reassigned {reassignments_made} cells based on neighborhood consensus")
    
    # Recalculate silhouette scores after reassignment
    new_labels = Zhang_dataset.obs['Consensus_annotation_detailed_refined'].astype('category').cat.codes
    new_silhouette_scores = silhouette_samples(X_embed, new_labels)
    silhouette_avg_corrected = silhouette_score(X_embed, new_labels)
    
    # Store corrected scores
    Zhang_dataset.obs['silhouette_score_corrected'] = new_silhouette_scores
    
    print(f"\n=== REASSIGNMENT RESULTS ===")
    print(f"Original average silhouette: {silhouette_avg:.3f}")
    print(f"Refined average silhouette: {silhouette_avg_corrected:.3f}")
    print(f"Improvement: {silhouette_avg_corrected - silhouette_avg:.3f}")
    
    print(f"Original negative silhouette cells: {negative_silhouette_mask.sum()}")
    print(f"Refined negative silhouette cells: {(new_silhouette_scores < 0).sum()}")
    
    # Show what changes were made
    if reassignments_made > 0:
        changes_mask = (Zhang_dataset.obs['Consensus_annotation_detailed'] != 
                       Zhang_dataset.obs['Consensus_annotation_detailed_refined'])
        changes = Zhang_dataset.obs[changes_mask]
        
        print(f"\n=== SPECIFIC REASSIGNMENTS ===")
        change_summary = changes.groupby([
            'Consensus_annotation_detailed', 
            'Consensus_annotation_detailed_refined'
        ]).size().reset_index(name='count')
        
        for _, row in change_summary.iterrows():
            print(f"{row['Consensus_annotation_detailed']} -> {row['Consensus_annotation_detailed_refined']}: {row['count']} cells")

else:
    print("No cells with very poor silhouette scores found.")
    # Create corrected scores column that's identical to original
    Zhang_dataset.obs['silhouette_score_corrected'] = Zhang_dataset.obs['silhouette_score'].copy()
    silhouette_avg_corrected = silhouette_avg

# Create a reassignment status column for visualization
reassignment_mask = (Zhang_dataset.obs['Consensus_annotation_detailed'] != 
                    Zhang_dataset.obs['Consensus_annotation_detailed_refined'])
Zhang_dataset.obs['reassignment_status'] = 'Unchanged'
Zhang_dataset.obs.loc[reassignment_mask, 'reassignment_status'] = 'Reassigned'

# Final summary
print(f"\n=== FINAL SUMMARY ===")
print(f"Total cells: {len(Zhang_dataset)}")
print(f"Cells reassigned: {reassignment_mask.sum()}")
print(f"Final cell type distribution:")
final_counts = Zhang_dataset.obs['Consensus_annotation_detailed_refined'].value_counts()
print(final_counts)

# Plot comprehensive analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Original annotations
sc.pl.embedding(Zhang_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_umap',
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,0])
axes[0,0].set_title('Original Smoothed Annotations', fontsize=14, fontweight='bold')

# Plot 2: Refined annotations
sc.pl.embedding(Zhang_dataset, 
                color='Consensus_annotation_detailed_refined', 
                basis='X_umap',
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,1])
axes[0,1].set_title('Silhouette-Refined Annotations', fontsize=14, fontweight='bold')

# Plot 3: Reassignment status
sc.pl.embedding(Zhang_dataset, 
                color='reassignment_status', 
                basis='X_umap',
                palette={'Unchanged': 'lightgray', 'Reassigned': 'red'},
                add_outline=False,
                legend_loc='right margin', 
                frameon=False,
                show=False,
                ax=axes[1,0])
axes[1,0].set_title('Reassignment Status', fontsize=14, fontweight='bold')

# Plot 4: Corrected silhouette scores
sc.pl.embedding(Zhang_dataset, 
                color='silhouette_score_corrected', 
                basis='X_umap',
                color_map='RdBu_r',
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[1,1])
axes[1,1].set_title('Silhouette Scores (Refined)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(figures_path + "/Zhang_dataset_silhouette_refinement_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

# Additional histogram comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# Original silhouette distribution
ax1.hist(sample_silhouette_values, bins=50, alpha=0.7, edgecolor='black', color='lightblue')
ax1.axvline(x=0, color='red', linestyle='--', label='Silhouette = 0')
ax1.set_xlabel('Silhouette Score')
ax1.set_ylabel('Number of Cells')
ax1.set_title(f'Original Silhouette Distribution\n(Avg: {silhouette_avg:.3f})')
ax1.legend()

# Refined silhouette distribution
ax2.hist(Zhang_dataset.obs['silhouette_score_corrected'], bins=50, alpha=0.7, edgecolor='black', color='lightgreen')
ax2.axvline(x=0, color='red', linestyle='--', label='Silhouette = 0')
ax2.set_xlabel('Silhouette Score')
ax2.set_ylabel('Number of Cells')
ax2.set_title(f'Refined Silhouette Distribution\n(Avg: {silhouette_avg_corrected:.3f})')
ax2.legend()

plt.tight_layout()
plt.savefig(figures_path + "/Zhang_dataset_silhouette_distribution_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Remove unused categories
Zhang_dataset.obs['Consensus_annotation_simplified'] = Zhang_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Zhang_dataset.obs['Consensus_annotation_detailed_refined'] = Zhang_dataset.obs['Consensus_annotation_detailed_refined'].cat.remove_unused_categories()

In [ ]:
Zhang_dataset_normalized = Zhang_dataset.copy()
Zhang_dataset_normalized.X = SCUtils.Protein_normalization(Zhang_dataset_normalized.X)
sc.tl.rank_genes_groups(Zhang_dataset_normalized, 'Consensus_annotation_detailed_refined', method='wilcoxon')
sc.pl.rank_genes_groups(Zhang_dataset_normalized, n_genes=10, sharey=False, ncols = 3, fontsize = 14)

plt.savefig(figures_path + "/Zhang_dataset_top10_markers.png", dpi=300, bbox_inches='tight')

In [ ]:
AveragedExpression = grouped_obs_mean(Zhang_dataset_normalized, 'Consensus_annotation_detailed_refined')
df = pd.DataFrame(AveragedExpression)

In [ ]:
# Compute the correlation matrix
corr = df.corr(method='pearson')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(235, 15, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
heatmap = sns.heatmap(corr, mask=mask, cmap=cmap, annot=True, 
                        square=True, linewidths=.6, cbar_kws={"shrink": 1},
                        annot_kws={"fontsize":5})

heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12)

plt.savefig(figures_path + "/Zhang_dataset_correlation_heatmap.png", dpi=300, bbox_inches='tight')

In [ ]:
Zhang_dataset.obs['Consensus_annotation_broad_final'] = 'Mature'

categories = ['Immature','Mature']

Zhang_dataset.obs['Consensus_annotation_broad_final'] = pd.Categorical(Zhang_dataset.obs['Consensus_annotation_broad_final'], categories=categories)
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['HSC','MPP', 'LMPP', 'EoBaMaP', 'Pre-Pro-B', 'Pro-B', 'GMP','MkP','ErP','MEP']), 'Consensus_annotation_broad_final'] = 'Immature'

In [ ]:
Zhang_dataset.obs['Consensus_annotation_simplified_final'] = ''

categories = ['HSPC', 'Monocyte', 'CD4 T', 'CD8 T', 'Erythroid', 'B', 'cDC', 'pDC', 'NK', 'Macrophage', 'Stroma', 'Myeloid', 'ILC', 'Doublet', 'Other T', 'Plasma']

Zhang_dataset.obs['Consensus_annotation_simplified_final'] = pd.Categorical(Zhang_dataset.obs['Consensus_annotation_simplified_final'], categories=categories)
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['HSC','MPP', 'LMPP', 'EoBaMaP', 'Pre-Pro-B', 'Pro-B', 'GMP','MkP','ErP','MEP']), 'Consensus_annotation_simplified_final'] = 'HSPC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD14 Mono', 'CD16 Mono']), 'Consensus_annotation_simplified_final'] = 'Monocyte'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['NK CD56 dim', 'NK CD56 bright']), 'Consensus_annotation_simplified_final'] = 'NK'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD4 T Naive', 'CD4 T Memory', 'Treg']), 'Consensus_annotation_simplified_final'] = 'CD4 T'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD8 T Naive', 'CD8 T Memory', 'MAIT']), 'Consensus_annotation_simplified_final'] = 'CD8 T'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['B Naive', 'B Memory', 'Immature B', 'Pre-B']), 'Consensus_annotation_simplified_final'] = 'B'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['cDC1', 'cDC2']), 'Consensus_annotation_simplified_final'] = 'cDC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Erythroblast', 'ErP']), 'Consensus_annotation_simplified_final'] = 'Erythroid'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Myeloid progenitor']), 'Consensus_annotation_simplified_final'] = 'Myeloid'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'] == 'pDC', 'Consensus_annotation_simplified_final'] = 'pDC'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Gamma delta T']), 'Consensus_annotation_simplified_final'] = 'Other T'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'] == 'Macrophage', 'Consensus_annotation_simplified_final'] = 'Macrophage'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'] == 'Stroma', 'Consensus_annotation_simplified_final'] = 'Stroma'
Zhang_dataset.obs.loc[Zhang_dataset.obs['Consensus_annotation_detailed_refined'] == 'Plasma', 'Consensus_annotation_simplified_final'] = 'Plasma'

In [ ]:
Zhang_dataset.obs['Consensus_annotation_detailed_final'] = Zhang_dataset.obs['Consensus_annotation_detailed_refined']

In [ ]:
# Remove Gamma delta T cells from the final dataset
print(f"Before removing Gamma delta T cells: {len(Zhang_dataset)} cells")
print("Cell type counts before:")
print(Zhang_dataset.obs['Consensus_annotation_detailed_final'].value_counts())

# Create mask to exclude Gamma delta T cells
mask_not_gdt = ~(Zhang_dataset.obs['Consensus_annotation_detailed_final'] == 'Gamma delta T')

# Filter the dataset
Zhang_dataset = Zhang_dataset[mask_not_gdt, :].copy()

# Remove unused categories from all annotation columns
Zhang_dataset.obs['Consensus_annotation_detailed_final'] = Zhang_dataset.obs['Consensus_annotation_detailed_final'].cat.remove_unused_categories()
Zhang_dataset.obs['Consensus_annotation_simplified_final'] = Zhang_dataset.obs['Consensus_annotation_simplified_final'].cat.remove_unused_categories()
Zhang_dataset.obs['Consensus_annotation_broad_final'] = Zhang_dataset.obs['Consensus_annotation_broad_final'].cat.remove_unused_categories()

print(f"After removing Gamma delta T cells: {len(Zhang_dataset)} cells")
print("Cell type counts after:")
print(Zhang_dataset.obs['Consensus_annotation_detailed_final'].value_counts())

# Plot updated annotations
sc.pl.embedding(Zhang_dataset, 
                color='Consensus_annotation_detailed_final', 
                basis='X_umap',
                legend_loc='on data',
                legend_fontsize=4,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

plt.tight_layout()
plt.show()

In [ ]:
# Remove unused categories
Zhang_dataset.obs['Consensus_annotation_simplified_final'] = Zhang_dataset.obs['Consensus_annotation_simplified_final'].cat.remove_unused_categories()
Zhang_dataset.obs['Consensus_annotation_detailed_final'] = Zhang_dataset.obs['Consensus_annotation_detailed_final'].cat.remove_unused_categories()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Zhang_dataset, 
                color='Consensus_annotation_broad_final', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Zhang X. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus broad annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Zhang_dataset_final_consensus_annotation_broad_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Zhang_dataset, 
                color='Consensus_annotation_simplified_final', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Zhang X. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus simplified annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Zhang_dataset_final_consensus_annotation_simplified_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Zhang_dataset, 
                color='Consensus_annotation_detailed_final', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Zhang X. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Zhang_dataset_final_Consensus_annotation_detailed_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

### Hao dataset

In [ ]:
# Remove unused categories
Hao_dataset.obs['Consensus_annotation_simplified'] = Hao_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Hao_dataset.obs['Consensus_annotation_detailed'] = Hao_dataset.obs['Consensus_annotation_detailed'].cat.remove_unused_categories()

In [ ]:
# Keep only the cells that are not labelled as 'Doublet'
Hao_dataset = Hao_dataset[Hao_dataset.obs['celltype.l2'] != 'Doublet']

In [ ]:
counts = Hao_dataset.obs['Consensus_annotation_detailed'].value_counts()
print(counts)

In [ ]:
filtered_categories = counts[counts >= 10].index
Hao_dataset = Hao_dataset[Hao_dataset.obs[Hao_dataset.obs['Consensus_annotation_detailed'].isin(filtered_categories)].index, :]

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed' in Hao_dataset.uns:
    del Hao_dataset.uns['Consensus_annotation_detailed']

# Plot UMAP with color
sc.pl.embedding(Hao_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_wnn.umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Hao Y. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation - smoothed', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
sc.pl.embedding(Hao_dataset, 
                color='celltype.l2', 
                basis='X_wnn.umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Hao et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation', fontsize=8, y=0.925,
            color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
# Get current categories and safely add new ones
current_categories = list(Hao_dataset.obs['Consensus_annotation_simplified'].cat.categories)

# Add new categories only if they don't already exist
new_cats_to_add = []
for cat in ['Erythroid', 'Other T']:
    if cat not in current_categories:
        new_cats_to_add.append(cat)

if new_cats_to_add:
    all_categories = current_categories + new_cats_to_add
    Hao_dataset.obs['Consensus_annotation_simplified'] = Hao_dataset.obs['Consensus_annotation_simplified'].cat.set_categories(all_categories)

# Get current categories and safely add new ones for detailed annotation
current_categories = list(Hao_dataset.obs['Consensus_annotation_detailed'].cat.categories)

# Add new categories only if they don't already exist
new_cats_to_add = []
for cat in ['Erythroblast', 'Transitional B', 'MAIT', 'Gamma delta T', 'cDC2', 'MPP', 'Platelet', 'Double negative T']:
    if cat not in current_categories:
        new_cats_to_add.append(cat)

if new_cats_to_add:
    all_categories = current_categories + new_cats_to_add
    Hao_dataset.obs['Consensus_annotation_detailed'] = Hao_dataset.obs['Consensus_annotation_detailed'].cat.set_categories(all_categories)

# Now proceed with the assignments
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'EoBaMaP', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'MAIT', 'Consensus_annotation_simplified'] = 'CD8 T'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'MAIT', 'Consensus_annotation_detailed'] = 'MAIT'

Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'Eryth', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'Eryth', 'Consensus_annotation_simplified'] = 'Erythroid'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'Eryth', 'Consensus_annotation_detailed'] = 'Erythroblast'

Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'gdT', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'gdT', 'Consensus_annotation_simplified'] = 'CD8 T'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'gdT', 'Consensus_annotation_detailed'] = 'Gamma delta T'

Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'dnT', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'dnT', 'Consensus_annotation_simplified'] = 'Other T'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'dnT', 'Consensus_annotation_detailed'] = 'Double negative T'

Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'ASDC_mDC', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'ASDC_mDC', 'Consensus_annotation_simplified'] = 'cDC'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'ASDC_mDC', 'Consensus_annotation_detailed'] = 'cDC2'

Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'HSPC', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'HSPC', 'Consensus_annotation_simplified'] = 'HSPC'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'HSPC', 'Consensus_annotation_detailed'] = 'MPP'

Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'CD8 TEM', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'CD8 TEM', 'Consensus_annotation_simplified'] = 'CD8 T'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'CD8 TEM', 'Consensus_annotation_detailed'] = 'CD8 T Memory'

Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'CD4 TCM', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'CD4 TCM', 'Consensus_annotation_simplified'] = 'CD4 T'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l2'] == 'CD4 TCM', 'Consensus_annotation_detailed'] = 'CD4 T Memory'

Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'Platelet', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'Platelet', 'Consensus_annotation_simplified'] = 'Erythroid'
Hao_dataset.obs.loc[Hao_dataset.obs['celltype.l3'] == 'Platelet', 'Consensus_annotation_detailed'] = 'Platelet'

Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed'] == 'Immature B', 'Consensus_annotation_broad'] = 'Mature'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed'] == 'Immature B', 'Consensus_annotation_simplified'] = 'B'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed'] == 'Immature B', 'Consensus_annotation_detailed'] = 'Immature B'

In [ ]:
# Remove unused categories
Hao_dataset.obs['Consensus_annotation_simplified'] = Hao_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Hao_dataset.obs['Consensus_annotation_detailed'] = Hao_dataset.obs['Consensus_annotation_detailed'].cat.remove_unused_categories()

In [ ]:
counts = Hao_dataset.obs['Consensus_annotation_detailed'].value_counts()
print(counts)

In [ ]:
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed'] == 'Progenitors', 'Consensus_annotation_broad'] = 'Immature'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed'] == 'Progenitors', 'Consensus_annotation_simplified'] = 'cDC'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed'] == 'Progenitors', 'Consensus_annotation_detailed'] = 'cDC2'

In [ ]:
counts = Hao_dataset.obs['Consensus_annotation_detailed'].value_counts()
print(counts)

In [ ]:
# ...existing code...

from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Calculate silhouette scores for current annotations
print("Calculating silhouette scores for Hao dataset...")

# Use the WNN UMAP representation for silhouette analysis
X_embed = Hao_dataset.obsm['X_wnn.umap']
labels = Hao_dataset.obs['Consensus_annotation_detailed'].astype('category').cat.codes

# Calculate silhouette scores
silhouette_avg = silhouette_score(X_embed, labels)
sample_silhouette_values = silhouette_samples(X_embed, labels)

print(f"Average silhouette score: {silhouette_avg:.3f}")

# Add silhouette scores to the dataset
Hao_dataset.obs['silhouette_score'] = sample_silhouette_values

# Identify cells with negative silhouette scores
negative_silhouette_mask = sample_silhouette_values < 0
print(f"Number of cells with negative silhouette scores: {negative_silhouette_mask.sum()}")
print(f"Percentage of cells with negative silhouette scores: {negative_silhouette_mask.sum()/len(sample_silhouette_values)*100:.2f}%")

# Show distribution of silhouette scores by cell type
silhouette_by_type = Hao_dataset.obs.groupby('Consensus_annotation_detailed')['silhouette_score'].agg(['mean', 'std', 'min', 'max', 'count'])
print("\nSilhouette scores by cell type:")
print(silhouette_by_type.sort_values('mean'))

# Initialize refined annotations (start with original smoothed annotations)
Hao_dataset.obs['Consensus_annotation_detailed_refined'] = Hao_dataset.obs['Consensus_annotation_detailed'].copy()

# Perform silhouette-based reassignment
print("\n=== PERFORMING SILHOUETTE-BASED REASSIGNMENT ===")

# Identify cells with very poor silhouette scores (< -0.1)
very_poor_silhouette = Hao_dataset.obs['silhouette_score'] < -0.1

if very_poor_silhouette.sum() > 0:
    print(f"Found {very_poor_silhouette.sum()} cells with very poor silhouette scores (< -0.1)")
    
    # Fit nearest neighbors
    nn = NearestNeighbors(n_neighbors=30, metric='euclidean')
    nn.fit(X_embed)
    
    # Get indices of poorly assigned cells
    poor_indices = np.where(very_poor_silhouette)[0]
    
    reassignments_made = 0
    
    for idx in poor_indices:
        # Find neighbors for this cell
        distances, neighbor_indices = nn.kneighbors([X_embed[idx]])
        neighbor_indices = neighbor_indices[0][1:]  # Exclude the cell itself
        
        # Get annotations of neighbors
        neighbor_annotations = Hao_dataset.obs['Consensus_annotation_detailed'].iloc[neighbor_indices]

        # Find most common annotation among neighbors
        most_common = neighbor_annotations.mode()
        
        if len(most_common) > 0:
            new_annotation = most_common.iloc[0]
            current_annotation = Hao_dataset.obs['Consensus_annotation_detailed'].iloc[idx]
            
            # Only reassign if the most common neighbor annotation is different
            if new_annotation != current_annotation:
                # Check if at least 40% of neighbors have this annotation
                fraction = (neighbor_annotations == new_annotation).sum() / len(neighbor_annotations)
                
                if fraction >= 0.4:
                    Hao_dataset.obs.loc[Hao_dataset.obs.index[idx], 'Consensus_annotation_detailed_refined'] = new_annotation
                    reassignments_made += 1
    
    print(f"Reassigned {reassignments_made} cells based on neighborhood consensus")
    
    # Recalculate silhouette scores after reassignment
    new_labels = Hao_dataset.obs['Consensus_annotation_detailed_refined'].astype('category').cat.codes
    new_silhouette_scores = silhouette_samples(X_embed, new_labels)
    silhouette_avg_corrected = silhouette_score(X_embed, new_labels)
    
    # Store corrected scores
    Hao_dataset.obs['silhouette_score_corrected'] = new_silhouette_scores
    
    print(f"\n=== REASSIGNMENT RESULTS ===")
    print(f"Original average silhouette: {silhouette_avg:.3f}")
    print(f"Refined average silhouette: {silhouette_avg_corrected:.3f}")
    print(f"Improvement: {silhouette_avg_corrected - silhouette_avg:.3f}")
    
    print(f"Original negative silhouette cells: {negative_silhouette_mask.sum()}")
    print(f"Refined negative silhouette cells: {(new_silhouette_scores < 0).sum()}")
    
    # Show what changes were made
    if reassignments_made > 0:
        changes_mask = (Hao_dataset.obs['Consensus_annotation_detailed'] != 
                       Hao_dataset.obs['Consensus_annotation_detailed_refined'])
        changes = Hao_dataset.obs[changes_mask]
        
        print(f"\n=== SPECIFIC REASSIGNMENTS ===")
        change_summary = changes.groupby([
            'Consensus_annotation_detailed', 
            'Consensus_annotation_detailed_refined'
        ]).size().reset_index(name='count')
        
        for _, row in change_summary.iterrows():
            print(f"{row['Consensus_annotation_detailed']} -> {row['Consensus_annotation_detailed_refined']}: {row['count']} cells")

else:
    print("No cells with very poor silhouette scores found.")
    # Create corrected scores column that's identical to original
    Hao_dataset.obs['silhouette_score_corrected'] = Hao_dataset.obs['silhouette_score'].copy()
    silhouette_avg_corrected = silhouette_avg

# Create a reassignment status column for visualization
reassignment_mask = (Hao_dataset.obs['Consensus_annotation_detailed'] != 
                    Hao_dataset.obs['Consensus_annotation_detailed_refined'])
Hao_dataset.obs['reassignment_status'] = 'Unchanged'
Hao_dataset.obs.loc[reassignment_mask, 'reassignment_status'] = 'Reassigned'

# Final summary
print(f"\n=== FINAL SUMMARY ===")
print(f"Total cells: {len(Hao_dataset)}")
print(f"Cells reassigned: {reassignment_mask.sum()}")
print(f"Final cell type distribution:")
final_counts = Hao_dataset.obs['Consensus_annotation_detailed_refined'].value_counts()
print(final_counts)

# Plot comprehensive analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Original annotations
sc.pl.embedding(Hao_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_wnn.umap',
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,0])
axes[0,0].set_title('Original Smoothed Annotations', fontsize=14, fontweight='bold')

# Plot 2: Refined annotations
sc.pl.embedding(Hao_dataset, 
                color='Consensus_annotation_detailed_refined', 
                basis='X_wnn.umap',
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,1])
axes[0,1].set_title('Silhouette-Refined Annotations', fontsize=14, fontweight='bold')

# Plot 3: Reassignment status
sc.pl.embedding(Hao_dataset, 
                color='reassignment_status', 
                basis='X_wnn.umap',
                palette={'Unchanged': 'lightgray', 'Reassigned': 'red'},
                add_outline=False,
                legend_loc='right margin', 
                frameon=False,
                show=False,
                ax=axes[1,0])
axes[1,0].set_title('Reassignment Status', fontsize=14, fontweight='bold')

# Plot 4: Corrected silhouette scores
sc.pl.embedding(Hao_dataset, 
                color='silhouette_score_corrected', 
                basis='X_wnn.umap',
                color_map='RdBu_r',
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[1,1])
axes[1,1].set_title('Silhouette Scores (Refined)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(figures_path + "/Hao_dataset_silhouette_refinement_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

# Additional histogram comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# Original silhouette distribution
ax1.hist(sample_silhouette_values, bins=50, alpha=0.7, edgecolor='black', color='lightblue')
ax1.axvline(x=0, color='red', linestyle='--', label='Silhouette = 0')
ax1.set_xlabel('Silhouette Score')
ax1.set_ylabel('Number of Cells')
ax1.set_title(f'Original Silhouette Distribution\n(Avg: {silhouette_avg:.3f})')
ax1.legend()

# Refined silhouette distribution
ax2.hist(Hao_dataset.obs['silhouette_score_corrected'], bins=50, alpha=0.7, edgecolor='black', color='lightgreen')
ax2.axvline(x=0, color='red', linestyle='--', label='Silhouette = 0')
ax2.set_xlabel('Silhouette Score')
ax2.set_ylabel('Number of Cells')
ax2.set_title(f'Refined Silhouette Distribution\n(Avg: {silhouette_avg_corrected:.3f})')
ax2.legend()

plt.tight_layout()
plt.savefig(figures_path + "/Hao_dataset_silhouette_distribution_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Remove unused categories
Hao_dataset.obs['Consensus_annotation_simplified'] = Hao_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Hao_dataset.obs['Consensus_annotation_detailed_refined'] = Hao_dataset.obs['Consensus_annotation_detailed_refined'].cat.remove_unused_categories()

In [ ]:
Hao_dataset_normalized = Hao_dataset.copy()
Hao_dataset_normalized.X = SCUtils.Protein_normalization(Hao_dataset_normalized.X)
sc.tl.rank_genes_groups(Hao_dataset_normalized, 'Consensus_annotation_detailed_refined', method='wilcoxon')
sc.pl.rank_genes_groups(Hao_dataset_normalized, n_genes=10, sharey=False, ncols = 3, fontsize = 14)

plt.savefig(figures_path + "/Hao_dataset_top10_markers.png", dpi=300, bbox_inches='tight')

In [ ]:
AveragedExpression = grouped_obs_mean(Hao_dataset_normalized, 'Consensus_annotation_detailed_refined')
df = pd.DataFrame(AveragedExpression)

In [ ]:
# Compute the correlation matrix
corr = df.corr(method='pearson')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(235, 15, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
heatmap = sns.heatmap(corr, mask=mask, cmap=cmap, annot=True, 
                        square=True, linewidths=.6, cbar_kws={"shrink": 1},
                        annot_kws={"fontsize":5})

heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12)

plt.savefig(figures_path + "/Hao_dataset_correlation_heatmap.png", dpi=300, bbox_inches='tight')

In [ ]:
Hao_dataset.obs['Consensus_annotation_broad_final'] = 'Mature'

categories = ['Immature','Mature']

Hao_dataset.obs['Consensus_annotation_broad_final'] = pd.Categorical(Hao_dataset.obs['Consensus_annotation_broad_final'], categories=categories)
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['MPP']), 'Consensus_annotation_broad_final'] = 'Immature'

In [ ]:
Hao_dataset.obs['Consensus_annotation_simplified_final'] = ''

categories = ['HSPC', 'Monocyte', 'CD4 T', 'CD8 T', 'Erythroid', 'B', 'cDC', 'pDC', 'NK', 'ILC', 'Other T', 'Macrophage', 'Stroma', 'Myeloid', 'Doublet', 'Plasma']

Hao_dataset.obs['Consensus_annotation_simplified_final'] = pd.Categorical(Hao_dataset.obs['Consensus_annotation_simplified_final'], categories=categories)
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['MPP']), 'Consensus_annotation_simplified_final'] = 'HSPC'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD14 Mono', 'CD16 Mono']), 'Consensus_annotation_simplified_final'] = 'Monocyte'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['NK CD56 dim', 'NK CD56 bright']), 'Consensus_annotation_simplified_final'] = 'NK'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD4 T Naive', 'CD4 T Memory', 'Treg', 'CD4 CTL']), 'Consensus_annotation_simplified_final'] = 'CD4 T'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD8 T Naive', 'CD8 T Memory', 'MAIT']), 'Consensus_annotation_simplified_final'] = 'CD8 T'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Pre-B', 'B Naive', 'B Memory', 'Immature B']), 'Consensus_annotation_simplified_final'] = 'B'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Erythroblast', 'Platelet']), 'Consensus_annotation_simplified_final'] = 'Erythroid'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'] == 'ILC', 'Consensus_annotation_simplified_final'] = 'ILC'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Gamma delta T', 'Double negative T']), 'Consensus_annotation_simplified_final'] = 'Other T'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'].isin(['cDC1', 'cDC2']), 'Consensus_annotation_simplified_final'] = 'cDC'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'] == 'pDC', 'Consensus_annotation_simplified_final'] = 'pDC'
Hao_dataset.obs.loc[Hao_dataset.obs['Consensus_annotation_detailed_refined'] == 'Plasma', 'Consensus_annotation_simplified_final'] = 'Plasma'

In [ ]:
Hao_dataset.obs['Consensus_annotation_detailed_final'] = Hao_dataset.obs['Consensus_annotation_detailed_refined']

In [ ]:
# Remove unused categories
Hao_dataset.obs['Consensus_annotation_simplified_final'] = Hao_dataset.obs['Consensus_annotation_simplified_final'].cat.remove_unused_categories()
Hao_dataset.obs['Consensus_annotation_detailed_final'] = Hao_dataset.obs['Consensus_annotation_detailed_final'].cat.remove_unused_categories()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Hao_dataset, 
                color='Consensus_annotation_broad_final', 
                basis='X_wnn.umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Hao Y. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus broad annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Hao_dataset_final_consensus_annotation_broad_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Hao_dataset, 
                color='Consensus_annotation_simplified_final', 
                basis='X_wnn.umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Hao Y. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus simplified annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Hao_dataset_final_consensus_annotation_simplified_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Hao_dataset, 
                color='Consensus_annotation_detailed_final', 
                basis='X_wnn.umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Hao Y. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Hao_dataset_final_Consensus_annotation_detailed_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

### Triana dataset

In [ ]:
# Remove unused categories
Triana_dataset.obs['Consensus_annotation_simplified'] = Triana_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Triana_dataset.obs['Consensus_annotation_detailed'] = Triana_dataset.obs['Consensus_annotation_detailed'].cat.remove_unused_categories()

In [ ]:
counts = Triana_dataset.obs['Consensus_annotation_detailed'].value_counts()
print(counts)

In [ ]:
filtered_categories = counts[counts >= 10].index
Triana_dataset = Triana_dataset[Triana_dataset.obs[Triana_dataset.obs['Consensus_annotation_detailed'].isin(filtered_categories)].index, :]

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed_colors' in Triana_dataset.uns:
    del Triana_dataset.uns['Consensus_annotation_detailed_colors']

# Plot UMAP with color
sc.pl.embedding(Triana_dataset, 
                color='CellTypes', 
                basis='X_mofaumap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Triana S. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation - smoothed', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed_colors' in Triana_dataset.uns:
    del Triana_dataset.uns['Consensus_annotation_detailed_colors']

# Plot UMAP with color
sc.pl.embedding(Triana_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_mofaumap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Triana S. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation - smoothed', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
Triana_dataset.obs['Consensus_annotation_broad_tmp'] = Triana_dataset.obs['Consensus_annotation_broad'] 
Triana_dataset.obs['Consensus_annotation_simplified_tmp'] = Triana_dataset.obs['Consensus_annotation_simplified'] 
Triana_dataset.obs['Consensus_annotation_detailed_tmp'] = Triana_dataset.obs['Consensus_annotation_detailed'] 

# Get current categories and safely add new ones
current_categories = list(Triana_dataset.obs['Consensus_annotation_simplified'].cat.categories)

# Add new categories only if they don't already exist
new_cats_to_add = []
for cat in ['Erythroid', 'Myeloid']:  # Added 'Myeloid' here
    if cat not in current_categories:
        new_cats_to_add.append(cat)

if new_cats_to_add:
    all_categories = current_categories + new_cats_to_add
    Triana_dataset.obs['Consensus_annotation_simplified'] = Triana_dataset.obs['Consensus_annotation_simplified'].cat.set_categories(all_categories)

# Get current categories and safely add new ones for detailed annotation
current_categories = list(Triana_dataset.obs['Consensus_annotation_detailed'].cat.categories)

# Add new categories only if they don't already exist
new_cats_to_add = []
for cat in ['Erythroblast', 'Transitional B', 'MAIT', 'Gamma delta T', 'cDC2', 'GMP', 'Myeloid progenitor', 'MkP', 'Platelet', 'Myeloid', 'LMPP', 'Pre-Pro-B']:
    if cat not in current_categories:
        new_cats_to_add.append(cat)

if new_cats_to_add:
    all_categories = current_categories + new_cats_to_add
    Triana_dataset.obs['Consensus_annotation_detailed'] = Triana_dataset.obs['Consensus_annotation_detailed'].cat.set_categories(all_categories)

# Now proceed with the assignments
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Early promyelocytes', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Early promyelocytes', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Early promyelocytes', 'Consensus_annotation_detailed'] = 'GMP'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Lymphomyeloid prog', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Lymphomyeloid prog', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Lymphomyeloid prog', 'Consensus_annotation_detailed'] = 'LMPP'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Early erythroid progenitor', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Early erythroid progenitor', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Early erythroid progenitor', 'Consensus_annotation_detailed'] = 'MEP'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Late erythroid progenitor', 'Consensus_annotation_broad'] = 'Mature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Late erythroid progenitor', 'Consensus_annotation_simplified'] = 'Erythroid'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Late erythroid progenitor', 'Consensus_annotation_detailed'] = 'ErP'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Erythro-myeloid progenitors', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Erythro-myeloid progenitors', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Erythro-myeloid progenitors', 'Consensus_annotation_detailed'] = 'MPP'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Eosinophil-basophil-mast cell progenitors', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Eosinophil-basophil-mast cell progenitors', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Eosinophil-basophil-mast cell progenitors', 'Consensus_annotation_detailed'] = 'EoBaMaP'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'GammaDelta T cells', 'Consensus_annotation_broad'] = 'Mature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'GammaDelta T cells', 'Consensus_annotation_simplified'] = 'CD8 T'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'GammaDelta T cells', 'Consensus_annotation_detailed'] = 'Gamma delta T'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'HSCs & MPPs', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'HSCs & MPPs', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'HSCs & MPPs', 'Consensus_annotation_detailed'] = 'MPP'

Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'HSC', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'HSC', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'HSC', 'Consensus_annotation_detailed'] = 'HSC'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Megakaryocyte progenitors', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Megakaryocyte progenitors', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Megakaryocyte progenitors', 'Consensus_annotation_detailed'] = 'MPP'

Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'Pro-B', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'Pro-B', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'Pro-B', 'Consensus_annotation_detailed'] = 'MkP'

Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'MkP', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'MkP', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'MkP', 'Consensus_annotation_detailed'] = 'MkP'

Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'Pre-Pro-B', 'Consensus_annotation_broad'] = 'Immature'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'Pre-Pro-B', 'Consensus_annotation_simplified'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_tmp'] == 'Pre-Pro-B', 'Consensus_annotation_detailed'] = 'Pro-B'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Conventional dendritic cell 1', 'Consensus_annotation_broad'] = 'Mature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Conventional dendritic cell 1', 'Consensus_annotation_simplified'] = 'cDC'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Conventional dendritic cell 1', 'Consensus_annotation_detailed'] = 'cDC1'

Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Late promyelocytes', 'Consensus_annotation_broad'] = 'Mature'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Late promyelocytes', 'Consensus_annotation_simplified'] = 'Myeloid'
Triana_dataset.obs.loc[Triana_dataset.obs['CellTypes'] == 'Late promyelocytes', 'Consensus_annotation_detailed'] = 'Myeloid progenitor'

In [ ]:
# Extract LMPP cells from Triana dataset
lmpp_mask = Triana_dataset.obs['Consensus_annotation_detailed'] == 'LMPP'
lmpp_subset = Triana_dataset[lmpp_mask].copy()

print(f"Number of LMPP cells: {lmpp_subset.n_obs}")
print(f"Original clusters containing LMPP: {lmpp_subset.obs['CellTypes'].unique()}")

# Check distribution of original cell types within LMPP
print("\nDistribution of original CellTypes within LMPP:")
print(lmpp_subset.obs['CellTypes'].value_counts())

# Perform subclustering on LMPP cells
sc.pp.neighbors(lmpp_subset, use_rep="X_mofaumap", n_neighbors=15, metric='euclidean', random_state=42)
sc.tl.leiden(lmpp_subset, resolution=0.5, random_state=42, key_added='lmpp_subclusters')

# Create UMAP for the LMPP subset
sc.tl.umap(lmpp_subset, random_state=42, min_dist=0.3)

# Plot the subclusters
sc.pl.embedding(lmpp_subset, 
                color='lmpp_subclusters', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.set_title('LMPP Subclustering', fontsize=12, fontweight='bold', y=1.1)

plt.show()

# First, add 'Pre-Pro-B' to the categories if not already present
if 'Pre-Pro-B' not in lmpp_subset.obs['Consensus_annotation_detailed'].cat.categories:
    lmpp_subset.obs['Consensus_annotation_detailed'] = lmpp_subset.obs['Consensus_annotation_detailed'].cat.add_categories(['Pre-Pro-B'])

# Also add to the main Triana_dataset categories if not already present
if 'Pre-Pro-B' not in Triana_dataset.obs['Consensus_annotation_detailed'].cat.categories:
    Triana_dataset.obs['Consensus_annotation_detailed'] = Triana_dataset.obs['Consensus_annotation_detailed'].cat.add_categories(['Pre-Pro-B'])

# Reassign subcluster 4 to Pre-Pro-B
cluster_4_cells = lmpp_subset.obs[lmpp_subset.obs['lmpp_subclusters'] == '4'].index
print(f"\nReassigning {len(cluster_4_cells)} cells from LMPP subcluster 4 to Pre-Pro-B")

# Update the consensus annotation in both the subset and main dataset
lmpp_subset.obs.loc[cluster_4_cells, 'Consensus_annotation_detailed'] = 'Pre-Pro-B'
Triana_dataset.obs.loc[cluster_4_cells, 'Consensus_annotation_detailed'] = 'Pre-Pro-B'

# Print confirmation
print(f"Updated {len(cluster_4_cells)} cells to Pre-Pro-B annotation")

# Plot the subclusters again to show the change
sc.pl.embedding(lmpp_subset, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.set_title('LMPP Subclustering - Updated Annotations', fontsize=12, fontweight='bold', y=1.1)

plt.show()

# Show updated distribution
print("\nUpdated distribution of consensus annotations within LMPP subset:")
print(lmpp_subset.obs['Consensus_annotation_detailed'].value_counts())

# Check original annotations within each subcluster
print("\nOriginal CellTypes per subcluster:")
for cluster in sorted(lmpp_subset.obs['lmpp_subclusters'].unique()):
    cluster_cells = lmpp_subset.obs[lmpp_subset.obs['lmpp_subclusters'] == cluster]
    print(f"\nSubcluster {cluster}:")
    print(cluster_cells['CellTypes'].value_counts())

In [ ]:
counts = Triana_dataset.obs['Consensus_annotation_detailed'].value_counts()
print(counts)

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Calculate silhouette scores for current annotations
print("Calculating silhouette scores for Triana dataset...")

# Use the harmony-corrected PCA representation for silhouette analysis
X_embed = Triana_dataset.obsm['X_mofaumap']
labels = Triana_dataset.obs['Consensus_annotation_detailed'].astype('category').cat.codes

# Calculate silhouette scores
silhouette_avg = silhouette_score(X_embed, labels)
sample_silhouette_values = silhouette_samples(X_embed, labels)

print(f"Average silhouette score: {silhouette_avg:.3f}")

# Add silhouette scores to the dataset
Triana_dataset.obs['silhouette_score'] = sample_silhouette_values

# Identify cells with negative silhouette scores
negative_silhouette_mask = sample_silhouette_values < 0
print(f"Number of cells with negative silhouette scores: {negative_silhouette_mask.sum()}")
print(f"Percentage of cells with negative silhouette scores: {negative_silhouette_mask.sum()/len(sample_silhouette_values)*100:.2f}%")

# Show distribution of silhouette scores by cell type
silhouette_by_type = Triana_dataset.obs.groupby('Consensus_annotation_detailed')['silhouette_score'].agg(['mean', 'std', 'min', 'max', 'count'])
print("\nSilhouette scores by cell type:")
print(silhouette_by_type.sort_values('mean'))

# Initialize refined annotations (start with original smoothed annotations)
Triana_dataset.obs['Consensus_annotation_detailed_refined'] = Triana_dataset.obs['Consensus_annotation_detailed'].copy()

# Perform silhouette-based reassignment
print("\n=== PERFORMING SILHOUETTE-BASED REASSIGNMENT ===")

# Identify cells with very poor silhouette scores (< -0.1)
very_poor_silhouette = Triana_dataset.obs['silhouette_score'] < -0.1

if very_poor_silhouette.sum() > 0:
    print(f"Found {very_poor_silhouette.sum()} cells with very poor silhouette scores (< -0.1)")
    
    # Fit nearest neighbors
    nn = NearestNeighbors(n_neighbors=30, metric='euclidean')
    nn.fit(X_embed)
    
    # Get indices of poorly assigned cells
    poor_indices = np.where(very_poor_silhouette)[0]
    
    reassignments_made = 0
    
    for idx in poor_indices:
        # Find neighbors for this cell
        distances, neighbor_indices = nn.kneighbors([X_embed[idx]])
        neighbor_indices = neighbor_indices[0][1:]  # Exclude the cell itself
        
        # Get annotations of neighbors
        neighbor_annotations = Triana_dataset.obs['Consensus_annotation_detailed'].iloc[neighbor_indices]

        # Find most common annotation among neighbors
        most_common = neighbor_annotations.mode()
        
        if len(most_common) > 0:
            new_annotation = most_common.iloc[0]
            current_annotation = Triana_dataset.obs['Consensus_annotation_detailed'].iloc[idx]
            
            # Only reassign if the most common neighbor annotation is different
            if new_annotation != current_annotation:
                # Check if at least 40% of neighbors have this annotation
                fraction = (neighbor_annotations == new_annotation).sum() / len(neighbor_annotations)
                
                if fraction >= 0.4:
                    Triana_dataset.obs.loc[Triana_dataset.obs.index[idx], 'Consensus_annotation_detailed_refined'] = new_annotation
                    reassignments_made += 1
    
    print(f"Reassigned {reassignments_made} cells based on neighborhood consensus")
    
    # Recalculate silhouette scores after reassignment
    new_labels = Triana_dataset.obs['Consensus_annotation_detailed_refined'].astype('category').cat.codes
    new_silhouette_scores = silhouette_samples(X_embed, new_labels)
    silhouette_avg_corrected = silhouette_score(X_embed, new_labels)
    
    # Store corrected scores
    Triana_dataset.obs['silhouette_score_corrected'] = new_silhouette_scores
    
    print(f"\n=== REASSIGNMENT RESULTS ===")
    print(f"Original average silhouette: {silhouette_avg:.3f}")
    print(f"Refined average silhouette: {silhouette_avg_corrected:.3f}")
    print(f"Improvement: {silhouette_avg_corrected - silhouette_avg:.3f}")
    
    print(f"Original negative silhouette cells: {negative_silhouette_mask.sum()}")
    print(f"Refined negative silhouette cells: {(new_silhouette_scores < 0).sum()}")
    
    # Show what changes were made
    if reassignments_made > 0:
        changes_mask = (Triana_dataset.obs['Consensus_annotation_detailed'] != 
                       Triana_dataset.obs['Consensus_annotation_detailed_refined'])
        changes = Triana_dataset.obs[changes_mask]
        
        print(f"\n=== SPECIFIC REASSIGNMENTS ===")
        change_summary = changes.groupby([
            'Consensus_annotation_detailed', 
            'Consensus_annotation_detailed_refined'
        ]).size().reset_index(name='count')
        
        for _, row in change_summary.iterrows():
            print(f"{row['Consensus_annotation_detailed']} -> {row['Consensus_annotation_detailed_refined']}: {row['count']} cells")

else:
    print("No cells with very poor silhouette scores found.")
    # Create corrected scores column that's identical to original
    Triana_dataset.obs['silhouette_score_corrected'] = Triana_dataset.obs['silhouette_score'].copy()
    silhouette_avg_corrected = silhouette_avg

# Create a reassignment status column for visualization
reassignment_mask = (Triana_dataset.obs['Consensus_annotation_detailed'] != 
                    Triana_dataset.obs['Consensus_annotation_detailed_refined'])
Triana_dataset.obs['reassignment_status'] = 'Unchanged'
Triana_dataset.obs.loc[reassignment_mask, 'reassignment_status'] = 'Reassigned'

# Final summary
print(f"\n=== FINAL SUMMARY ===")
print(f"Total cells: {len(Triana_dataset)}")
print(f"Cells reassigned: {reassignment_mask.sum()}")
print(f"Final cell type distribution:")
final_counts = Triana_dataset.obs['Consensus_annotation_detailed_refined'].value_counts()
print(final_counts)

# Plot comprehensive analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Original annotations
sc.pl.embedding(Triana_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_mofaumap',
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,0])
axes[0,0].set_title('Original Smoothed Annotations', fontsize=14, fontweight='bold')

# Plot 2: Refined annotations
sc.pl.embedding(Triana_dataset, 
                color='Consensus_annotation_detailed_refined', 
                basis='X_mofaumap',
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,1])
axes[0,1].set_title('Silhouette-Refined Annotations', fontsize=14, fontweight='bold')

# Plot 3: Reassignment status
sc.pl.embedding(Triana_dataset, 
                color='reassignment_status', 
                basis='X_mofaumap',
                palette={'Unchanged': 'lightgray', 'Reassigned': 'red'},
                add_outline=False,
                legend_loc='right margin', 
                frameon=False,
                show=False,
                ax=axes[1,0])
axes[1,0].set_title('Reassignment Status', fontsize=14, fontweight='bold')

# Plot 4: Corrected silhouette scores
sc.pl.embedding(Triana_dataset, 
                color='silhouette_score_corrected', 
                basis='X_mofaumap',
                color_map='RdBu_r',
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[1,1])
axes[1,1].set_title('Silhouette Scores (Refined)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(figures_path + "/Triana_dataset_silhouette_refinement_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

# Additional histogram comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# Original silhouette distribution
ax1.hist(sample_silhouette_values, bins=50, alpha=0.7, edgecolor='black', color='lightblue')
ax1.axvline(x=0, color='red', linestyle='--', label='Silhouette = 0')
ax1.set_xlabel('Silhouette Score')
ax1.set_ylabel('Number of Cells')
ax1.set_title(f'Original Silhouette Distribution\n(Avg: {silhouette_avg:.3f})')
ax1.legend()

# Refined silhouette distribution
ax2.hist(Triana_dataset.obs['silhouette_score_corrected'], bins=50, alpha=0.7, edgecolor='black', color='lightgreen')
ax2.axvline(x=0, color='red', linestyle='--', label='Silhouette = 0')
ax2.set_xlabel('Silhouette Score')
ax2.set_ylabel('Number of Cells')
ax2.set_title(f'Refined Silhouette Distribution\n(Avg: {silhouette_avg_corrected:.3f})')
ax2.legend()

plt.tight_layout()
plt.savefig(figures_path + "/Triana_dataset_silhouette_distribution_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Remove unused categories
Triana_dataset.obs['Consensus_annotation_simplified'] = Triana_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Triana_dataset.obs['Consensus_annotation_detailed_refined'] = Triana_dataset.obs['Consensus_annotation_detailed_refined'].cat.remove_unused_categories()

In [ ]:
Triana_dataset_normalized = Triana_dataset.copy()
Triana_dataset_normalized.X = SCUtils.Protein_normalization(Triana_dataset_normalized.X)
sc.tl.rank_genes_groups(Triana_dataset_normalized, 'Consensus_annotation_detailed_refined', method='wilcoxon')
sc.pl.rank_genes_groups(Triana_dataset_normalized, n_genes=10, sharey=False, ncols = 3, fontsize = 14)

plt.savefig(figures_path + "/Triana_dataset_top10_markers.png", dpi=300, bbox_inches='tight')

In [ ]:
AveragedExpression = grouped_obs_mean(Triana_dataset_normalized, 'Consensus_annotation_detailed_refined')
df = pd.DataFrame(AveragedExpression)

In [ ]:
# Compute the correlation matrix
corr = df.corr(method='pearson')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(235, 15, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
heatmap = sns.heatmap(corr, mask=mask, cmap=cmap, annot=True, 
                        square=True, linewidths=.6, cbar_kws={"shrink": 1},
                        annot_kws={"fontsize":5})

heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12)

plt.savefig(figures_path + "/Triana_dataset_correlation_heatmap.png", dpi=300, bbox_inches='tight')

In [ ]:
Triana_dataset.obs['Consensus_annotation_broad_final'] = 'Mature'

categories = ['Immature','Mature']

Triana_dataset.obs['Consensus_annotation_broad_final'] = pd.Categorical(Triana_dataset.obs['Consensus_annotation_broad_final'], categories=categories)
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['HSC', 'MPP', 'LMPP', 'EoBaMaP', 'MkP', 'MEP', 'Pre-Pro-B', 'Pro-B', 'GMP']), 'Consensus_annotation_broad_final'] = 'Immature'

In [ ]:
Triana_dataset.obs['Consensus_annotation_simplified_final'] = ''

categories = ['HSPC', 'Monocyte', 'CD4 T', 'CD8 T', 'Erythroid', 'B', 'cDC', 'pDC', 'NK', 'ILC', 'Stroma', 'Myeloid', 'Pro-B', 'Other T', 'Plasma']

Triana_dataset.obs['Consensus_annotation_simplified_final'] = pd.Categorical(Triana_dataset.obs['Consensus_annotation_simplified_final'], categories=categories)
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['HSC', 'MPP', 'LMPP', 'EoBaMaP', 'MkP', 'MEP', 'Pre-Pro-B', 'Pro-B', 'GMP']), 'Consensus_annotation_simplified_final'] = 'HSPC'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD14 Mono', 'CD16 Mono']), 'Consensus_annotation_simplified_final'] = 'Monocyte'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Myeloid progenitor']), 'Consensus_annotation_simplified_final'] = 'Myeloid'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['NK CD56 dim', 'NK CD56 bright']), 'Consensus_annotation_simplified_final'] = 'NK'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD4 T Naive', 'CD4 T Memory', 'Treg', 'CD4 CTL']), 'Consensus_annotation_simplified_final'] = 'CD4 T'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD8 T Naive', 'CD8 T Memory', 'MAIT']), 'Consensus_annotation_simplified_final'] = 'CD8 T'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['B Naive', 'B Memory', 'Immature B', 'Pre-B']), 'Consensus_annotation_simplified_final'] = 'B'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['ErP', 'Erythroblast']), 'Consensus_annotation_simplified_final'] = 'Erythroid'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['cDC1', 'cDC2']), 'Consensus_annotation_simplified_final'] = 'cDC'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Gamma delta T']), 'Consensus_annotation_simplified_final'] = 'Other T'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'] == 'pDC', 'Consensus_annotation_simplified_final'] = 'pDC'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'] == 'Stroma', 'Consensus_annotation_simplified_final'] = 'Stroma'
Triana_dataset.obs.loc[Triana_dataset.obs['Consensus_annotation_detailed_refined'] == 'Plasma', 'Consensus_annotation_simplified_final'] = 'Plasma'

In [ ]:
Triana_dataset.obs['Consensus_annotation_detailed_final'] = Triana_dataset.obs['Consensus_annotation_detailed_refined']

In [ ]:
# Extract B cells (Pro-B and Pre-Pro-B) from Triana dataset for clustering analysis
b_cell_types = ['Pro-B', 'Pre-Pro-B']
b_cell_mask = Triana_dataset.obs['Consensus_annotation_detailed_final'].isin(b_cell_types)
b_grouped_subset = Triana_dataset[b_cell_mask].copy()

print(f"Number of B cells extracted: {b_grouped_subset.n_obs}")
print(f"B cell types distribution:")
print(b_grouped_subset.obs['Consensus_annotation_detailed_final'].value_counts())

if len(b_grouped_subset) > 0:
    # Perform low-resolution clustering on B cells
    sc.pp.neighbors(b_grouped_subset, use_rep="X_mofaumap", n_neighbors=15, metric='euclidean', random_state=42)
    sc.tl.leiden(b_grouped_subset, resolution=0.3, random_state=42, key_added='low_res_clusters')
    
    # Create UMAP for the B cell subset
    sc.tl.umap(b_grouped_subset, random_state=42, min_dist=0.3)
    
    # Plot the clusters
    sc.pl.embedding(b_grouped_subset, 
                    color='low_res_clusters', 
                    basis='X_umap', 
                    legend_loc='on data', 
                    legend_fontsize=6,
                    legend_fontoutline=2,
                    add_outline=False,
                    frameon=False,
                    show=False)
    
    ax = plt.gca()
    ax.figure.set_size_inches(6, 5)
    ax.set_xlabel('UMAP 1', fontsize=12)
    ax.set_ylabel('UMAP 2', fontsize=12)
    ax.set_title('B Cell Clustering', fontsize=12, fontweight='bold', y=1.1)
    
    plt.show()
    
    # Plot original annotations for comparison
    sc.pl.embedding(b_grouped_subset, 
                    color='Consensus_annotation_detailed_final', 
                    basis='X_umap', 
                    legend_loc='on data', 
                    legend_fontsize=6,
                    legend_fontoutline=2,
                    add_outline=False,
                    frameon=False,
                    show=False)
    
    ax = plt.gca()
    ax.figure.set_size_inches(6, 5)
    ax.set_xlabel('UMAP 1', fontsize=12)
    ax.set_ylabel('UMAP 2', fontsize=12)
    ax.set_title('Original B Cell Annotations', fontsize=12, fontweight='bold', y=1.1)
    
    plt.show()
    
    # Show original annotations within each cluster for analysis
    print("\nOriginal annotations per cluster:")
    for cluster in sorted(b_grouped_subset.obs['low_res_clusters'].unique()):
        cluster_cells = b_grouped_subset.obs[b_grouped_subset.obs['low_res_clusters'] == cluster]
        print(f"\nCluster {cluster} ({len(cluster_cells)} cells):")
        print(cluster_cells['Consensus_annotation_detailed_final'].value_counts())
        
        # Calculate proportions
        proportions = cluster_cells['Consensus_annotation_detailed_final'].value_counts(normalize=True)
        print("Proportions:")
        for annotation, prop in proportions.items():
            print(f"  {annotation}: {prop:.2%}")
    
    print(f"\n" + "="*50)
    print("CLUSTERING ANALYSIS COMPLETE")
    print("="*50)
    print("Based on the cluster analysis above, decide which clusters to reassign:")
    print("- Look at the spatial distribution of clusters in the UMAP")
    print("- Consider the original annotation proportions within each cluster")
    print("- Use marker genes (if available) to guide decisions")
    print("\nRecommendation: Update the cluster assignment in the next cell")
    
else:
    print("No Pro-B or Pre-Pro-B cells found in the Triana dataset.")

In [ ]:
# DECISION POINT: Based on the clustering analysis above, specify which clusters to assign
# Update these assignments based on your analysis of the previous cell's output

# Cluster assignments based on analysis:
cluster_to_pre_pro_b = ['1', '0', '2', '6']  # Clusters to assign as Pre-Pro-B
cluster_to_pro_b = 'all_others'  # All other clusters assigned as Pro-B

print(f"Cluster assignment plan:")
print(f"Pre-Pro-B: clusters {cluster_to_pre_pro_b}")
if cluster_to_pro_b == 'all_others':
    other_clusters = [c for c in b_grouped_subset.obs['low_res_clusters'].unique() if c not in cluster_to_pre_pro_b]
    print(f"Pro-B: all other clusters {other_clusters}")
else:
    print(f"Pro-B: clusters {cluster_to_pro_b}")

# Proceed with refinement if b_grouped_subset exists and has data
if 'b_grouped_subset' in locals() and len(b_grouped_subset) > 0:
    # Create a new annotation column based on cluster assignments
    b_grouped_subset.obs['refined_annotation'] = b_grouped_subset.obs['Consensus_annotation_detailed_final'].copy()
    
    # Add the new categories first before assignment
    current_categories = b_grouped_subset.obs['refined_annotation'].cat.categories.tolist()
    new_categories = ['Pro-B', 'Pre-Pro-B']
    categories_to_add = [cat for cat in new_categories if cat not in current_categories]
    
    if categories_to_add:
        b_grouped_subset.obs['refined_annotation'] = b_grouped_subset.obs['refined_annotation'].cat.add_categories(categories_to_add)
    
    # Assign specific clusters to Pre-Pro-B
    for cluster in cluster_to_pre_pro_b:
        b_grouped_subset.obs.loc[b_grouped_subset.obs['low_res_clusters'] == cluster, 'refined_annotation'] = 'Pre-Pro-B'
    
    # Assign all other clusters to Pro-B
    other_clusters = [c for c in b_grouped_subset.obs['low_res_clusters'].unique() if c not in cluster_to_pre_pro_b]
    for cluster in other_clusters:
        b_grouped_subset.obs.loc[b_grouped_subset.obs['low_res_clusters'] == cluster, 'refined_annotation'] = 'Pro-B'
    
    # Remove unused categories
    b_grouped_subset.obs['refined_annotation'] = b_grouped_subset.obs['refined_annotation'].cat.remove_unused_categories()
    
    # Print the results
    print("\nRefined B cell annotation distribution:")
    print(b_grouped_subset.obs['refined_annotation'].value_counts())
    
    # Show the changes
    print("\nComparison of original vs refined B cell annotations:")
    b_comparison = pd.crosstab(
        b_grouped_subset.obs['Consensus_annotation_detailed_final'], 
        b_grouped_subset.obs['refined_annotation'],
        margins=True
    )
    print(b_comparison)
    
    # Apply the refined annotations back to the main Triana dataset
    print("\nUpdating main Triana dataset with refined B cell annotations...")
    
    # Get the indices of cells that were in the subset
    b_grouped_indices = b_grouped_subset.obs_names
    
    # Convert to regular strings to avoid categorical issues
    b_refined_values = b_grouped_subset.obs['refined_annotation'].astype(str)
    
    # Add new categories to the main dataset first
    new_b_categories = set(b_refined_values.unique()) - set(Triana_dataset.obs['Consensus_annotation_detailed_final'].cat.categories)
    if new_b_categories:
        print(f"Adding new B cell categories: {new_b_categories}")
        Triana_dataset.obs['Consensus_annotation_detailed_final'] = Triana_dataset.obs['Consensus_annotation_detailed_final'].cat.add_categories(list(new_b_categories))
    
    # Update the main dataset with refined annotations
    Triana_dataset.obs.loc[b_grouped_indices, 'Consensus_annotation_detailed_final'] = b_refined_values
    
    # Convert back to categorical if needed
    Triana_dataset.obs['Consensus_annotation_detailed_final'] = pd.Categorical(Triana_dataset.obs['Consensus_annotation_detailed_final'])
    
    # Verify the update
    print("Updated cell type counts in main Triana dataset:")
    print(Triana_dataset.obs['Consensus_annotation_detailed_final'].value_counts())
    
    # Check how many cells changed annotation
    original_pro_b = (b_grouped_subset.obs['Consensus_annotation_detailed_final'] == 'Pro-B').sum()
    original_pre_pro_b = (b_grouped_subset.obs['Consensus_annotation_detailed_final'] == 'Pre-Pro-B').sum()
    refined_pro_b = (b_grouped_subset.obs['refined_annotation'] == 'Pro-B').sum()
    refined_pre_pro_b = (b_grouped_subset.obs['refined_annotation'] == 'Pre-Pro-B').sum()
    
    print(f"\nB cell annotation changes summary:")
    print(f"Original Pro-B: {original_pro_b} → Refined Pro-B: {refined_pro_b} (change: {refined_pro_b - original_pro_b:+d})")
    print(f"Original Pre-Pro-B: {original_pre_pro_b} → Refined Pre-Pro-B: {refined_pre_pro_b} (change: {refined_pre_pro_b - original_pre_pro_b:+d})")
    
    # Visualize the refined annotations
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Plot original annotations
    sc.pl.umap(b_grouped_subset, 
               color='Consensus_annotation_detailed_final', 
               ax=axes[0], 
               show=False,
               title='Original B Cell Annotations')
    
    # Plot clusters with assignment info
    sc.pl.umap(b_grouped_subset, 
               color='low_res_clusters', 
               ax=axes[1], 
               show=False,
               title=f'Clusters (1,0,2,6 → Pre-Pro-B, others → Pro-B)')
    
    # Plot refined annotations
    sc.pl.umap(b_grouped_subset, 
               color='refined_annotation', 
               ax=axes[2], 
               show=False,
               title='Refined B Cell Annotations')
    
    plt.tight_layout()
    plt.show()
    
    print("\nMain Triana dataset has been updated with cluster-based refined B cell annotations!")

else:
    print("Error: B cell subset not found. Please run the clustering analysis first.")

In [ ]:
# Remove unused categories
Triana_dataset.obs['Consensus_annotation_simplified_final'] = Triana_dataset.obs['Consensus_annotation_simplified_final'].cat.remove_unused_categories()
Triana_dataset.obs['Consensus_annotation_detailed_final'] = Triana_dataset.obs['Consensus_annotation_detailed_final'].cat.remove_unused_categories()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Triana_dataset, 
                color='Consensus_annotation_broad_final', 
                basis='X_mofaumap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Triana S. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus broad annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Triana_dataset_final_consensus_annotation_broad_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Triana_dataset, 
                color='Consensus_annotation_simplified_final', 
                basis='X_mofaumap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Triana S. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus simplified annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Triana_dataset_final_consensus_annotation_simplified_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed_final_colors' in Triana_dataset.uns:
    del Triana_dataset.uns['Consensus_annotation_detailed_final_colors']

# Plot UMAP with color
sc.pl.embedding(Triana_dataset, 
                color='Consensus_annotation_detailed_final', 
                basis='X_mofaumap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Triana S. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Triana_dataset_final_Consensus_annotation_detailed_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

### Luecken dataset

In [ ]:
# Remove unused categories
Luecken_dataset.obs['Consensus_annotation_simplified'] = Luecken_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Luecken_dataset.obs['Consensus_annotation_detailed'] = Luecken_dataset.obs['Consensus_annotation_detailed'].cat.remove_unused_categories()

In [ ]:
counts = Luecken_dataset.obs['Consensus_annotation_detailed'].value_counts()

In [ ]:
print(counts)

In [ ]:
filtered_categories = counts[counts >= 10].index
Luecken_dataset = Luecken_dataset[Luecken_dataset.obs[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(filtered_categories)].index, :]

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Luecken_dataset, 
                color='cell_type', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Luecken M.D. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation - smoothed', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
# List the categories in the cell_type column
print("Cell type categories in Luecken dataset:")
print(list(Luecken_dataset.obs['cell_type'].cat.categories))
print(f"\nNumber of categories: {len(Luecken_dataset.obs['cell_type'].cat.categories)}")

# Also show value counts to see distribution
print("\nValue counts:")
print(Luecken_dataset.obs['cell_type'].value_counts())

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed_colors' in Luecken_dataset.uns:
    del Luecken_dataset.uns['Consensus_annotation_detailed_colors']

# Plot UMAP with color
sc.pl.embedding(Luecken_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Luecken M.D. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation - smoothed', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
# Get current categories and safely add new ones
current_categories = list(Luecken_dataset.obs['Consensus_annotation_simplified'].cat.categories)

# Add new categories only if they don't already exist
new_cats_to_add = []
for cat in ['Erythroid', 'Myeloid', 'Other T']:  # Added 'Myeloid' here
    if cat not in current_categories:
        new_cats_to_add.append(cat)

if new_cats_to_add:
    all_categories = current_categories + new_cats_to_add
    Luecken_dataset.obs['Consensus_annotation_simplified'] = Luecken_dataset.obs['Consensus_annotation_simplified'].cat.set_categories(all_categories)

# Get current categories and safely add new ones for detailed annotation
current_categories = list(Luecken_dataset.obs['Consensus_annotation_detailed'].cat.categories)

# Add new categories only if they don't already exist
new_cats_to_add = []
for cat in ['Erythroblast', 'Transitional B', 'MAIT', 'Gamma delta T', 'cDC2', 'MPP', 'Platelet', 'MkP', 'EoBaMaP', 'Myeloid progenitor', 'Neutrophil progenitor', 'pDC progenitor', 'Progenitors', 'Myeloid', 'Double negative T', 'ILC', 'cDC1', 'CD4 T', 'CD8 T', 'B', 'NK', 'Stroma']:
    if cat not in current_categories:
        new_cats_to_add.append(cat)

if new_cats_to_add:
    all_categories = current_categories + new_cats_to_add
    Luecken_dataset.obs['Consensus_annotation_detailed'] = Luecken_dataset.obs['Consensus_annotation_detailed'].cat.set_categories(all_categories)

# Now proceed with the assignments

Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['Proerythroblast', 'Erythroblast', 'Reticulocyte']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['Proerythroblast', 'Erythroblast', 'Reticulocyte']), 'Consensus_annotation_simplified'] = 'Erythroid'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['Proerythroblast', 'Erythroblast', 'Reticulocyte']), 'Consensus_annotation_detailed'] = 'Erythroblast'

Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['MK/E prog']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['MK/E prog']), 'Consensus_annotation_simplified'] = 'Erythroid'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['MK/E prog']), 'Consensus_annotation_detailed'] = 'ErP'

Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(['HSC', 'MkP', 'MEP']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(['HSC', 'MkP', 'MEP']), 'Consensus_annotation_simplified'] = 'Erythroid'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(['HSC', 'MkP', 'MEP']), 'Consensus_annotation_detailed'] = 'ErP'

Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(['LMPP']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(['LMPP']), 'Consensus_annotation_simplified'] = 'Myeloid'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(['LMPP']), 'Consensus_annotation_detailed'] = 'Myeloid progenitor'

Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['MAIT']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['MAIT']), 'Consensus_annotation_simplified'] = 'CD8 T'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['MAIT']), 'Consensus_annotation_detailed'] = 'MAIT'

Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['cDC1']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['cDC1']), 'Consensus_annotation_simplified'] = 'cDC'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['cDC1']), 'Consensus_annotation_detailed'] = 'cDC1'

Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['gdT TCRVD2+']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['gdT TCRVD2+']), 'Consensus_annotation_simplified'] = 'CD8 T'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['gdT TCRVD2+']), 'Consensus_annotation_detailed'] = 'Gamma delta T'

Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['dnT']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['dnT']), 'Consensus_annotation_simplified'] = 'Other T'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['dnT']), 'Consensus_annotation_detailed'] = 'Double negative T'

Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['CD4+ T activated IntegrinB7+', 'CD4+ T activated', 'T prog cycling']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['CD4+ T activated IntegrinB7+', 'CD4+ T activated', 'T prog cycling']), 'Consensus_annotation_simplified'] = 'CD4 T'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['CD4+ T activated IntegrinB7+', 'CD4+ T activated', 'T prog cycling']), 'Consensus_annotation_detailed'] = 'CD4 T Memory'

Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['CD4+ T Naive']), 'Consensus_annotation_broad'] = 'Mature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['CD4+ T Naive']), 'Consensus_annotation_simplified'] = 'CD4 T'
Luecken_dataset.obs.loc[Luecken_dataset.obs['cell_type'].isin(['CD4+ T Naive']), 'Consensus_annotation_detailed'] = 'CD4 T Naive'

Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(['CD45RO+', 'MPP', 'Pre-Pro-B', 'EoBaMaP', 'GMP']), 'Consensus_annotation_broad'] = 'Immature'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(['CD45RO+', 'MPP', 'Pre-Pro-B', 'EoBaMaP', 'GMP']), 'Consensus_annotation_simplified'] = 'HSPC'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed'].isin(['CD45RO+', 'MPP', 'Pre-Pro-B', 'EoBaMaP', 'GMP']), 'Consensus_annotation_detailed'] = 'MPP'


In [ ]:
# Plot UMAP with Consensus_annotation_detailed
sc.pl.embedding(Luecken_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(8, 6)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
# Extract ErP cells from Luecken dataset
erp_mask = Luecken_dataset.obs['Consensus_annotation_detailed'] == 'ErP'
erp_subset = Luecken_dataset[erp_mask].copy()

print(f"Number of ErP cells: {erp_subset.n_obs}")
print(f"Original clusters containing ErP: {erp_subset.obs['cell_type'].unique()}")

# Check distribution of original cell types within ErP
print("\nDistribution of original cell_type within ErP:")
print(erp_subset.obs['cell_type'].value_counts())

# Perform subclustering on ErP cells
sc.pp.neighbors(erp_subset, use_rep="X_pcahm", n_neighbors=15, metric='euclidean', random_state=42)
sc.tl.leiden(erp_subset, resolution=0.8, random_state=42, key_added='erp_subclusters')

# Create UMAP for the ErP subset
sc.tl.umap(erp_subset, random_state=42, min_dist=0.3)

# Plot the subclusters
sc.pl.embedding(erp_subset, 
                color='erp_subclusters', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.set_title('ErP Subclustering', fontsize=12, fontweight='bold', y=1.1)

plt.show()

# Plot original cell types for comparison
sc.pl.embedding(erp_subset, 
                color='cell_type', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.set_title('ErP Original Cell Types', fontsize=12, fontweight='bold', y=1.1)

plt.show()

# Analyze subclusters for potential MEP reassignment
print("\nAnalyzing subclusters for potential MEP reassignment:")
for cluster in sorted(erp_subset.obs['erp_subclusters'].unique()):
    cluster_cells = erp_subset.obs[erp_subset.obs['erp_subclusters'] == cluster]
    print(f"\nSubcluster {cluster} ({len(cluster_cells)} cells):")
    print(cluster_cells['cell_type'].value_counts())
    
    # Check if this cluster is enriched for MEP-like cells
    mep_like_cells = cluster_cells['cell_type'].isin(['MK/E prog', 'HSC/MPP']).sum()
    mep_fraction = mep_like_cells / len(cluster_cells)
    print(f"MEP-like fraction: {mep_fraction:.2f}")
    
    # Additional analysis - check for erythroid vs myeloid markers if available
    erythroid_cells = cluster_cells['cell_type'].isin(['MK/E prog', 'Proerythroblast', 'Erythroblast']).sum()
    erythroid_fraction = erythroid_cells / len(cluster_cells)
    print(f"Erythroid-like fraction: {erythroid_fraction:.2f}")

print("\n" + "="*50)
print("ANALYSIS COMPLETE")
print("="*50)
print("Based on the subcluster analysis above, decide which clusters to reassign:")
print("- Look for clusters with high MEP-like fraction")
print("- Consider clusters with 'MK/E prog' or 'HSC/MPP' cell types")
print("- Update the target_clusters list in the next script")

In [ ]:
# First, add all new categories if not already present
new_categories = ['Macrophage', 'MPP']

for cat in new_categories:
    if cat not in erp_subset.obs['Consensus_annotation_detailed'].cat.categories:
        erp_subset.obs['Consensus_annotation_detailed'] = erp_subset.obs['Consensus_annotation_detailed'].cat.add_categories([cat])

    if cat not in Luecken_dataset.obs['Consensus_annotation_detailed'].cat.categories:
        Luecken_dataset.obs['Consensus_annotation_detailed'] = Luecken_dataset.obs['Consensus_annotation_detailed'].cat.add_categories([cat])

# Also add 'Macrophage' to the simplified annotation categories if not present
if 'Macrophage' not in Luecken_dataset.obs['Consensus_annotation_simplified'].cat.categories:
    Luecken_dataset.obs['Consensus_annotation_simplified'] = Luecken_dataset.obs['Consensus_annotation_simplified'].cat.add_categories(['Macrophage'])

# Define reassignment mapping (keeping 0 and 1 as ErP)
reassignment_mapping = {
    '3': 'MPP'
}

print(f"Reassigning subclusters with the following mapping:")
for cluster, new_annotation in reassignment_mapping.items():
    print(f"  Subcluster {cluster} -> {new_annotation}")
print("  Subclusters 0 and 1 remain as ErP")

reassigned_cells_summary = {}

for cluster, new_annotation in reassignment_mapping.items():
    cluster_cells = erp_subset.obs[erp_subset.obs['erp_subclusters'] == cluster].index
    if len(cluster_cells) > 0:
        print(f"\nReassigning {len(cluster_cells)} cells from ErP subcluster {cluster} to {new_annotation}")
        
        # Update the consensus annotation in both the subset and main dataset
        erp_subset.obs.loc[cluster_cells, 'Consensus_annotation_detailed'] = new_annotation
        Luecken_dataset.obs.loc[cluster_cells, 'Consensus_annotation_detailed'] = new_annotation
        
        # Store for summary
        reassigned_cells_summary[new_annotation] = reassigned_cells_summary.get(new_annotation, [])
        reassigned_cells_summary[new_annotation].extend(cluster_cells)

total_reassigned = sum(len(cells) for cells in reassigned_cells_summary.values())
print(f"\nTotal updated {total_reassigned} cells with new annotations")
print(f"Subclusters 0 and 1 kept as ErP")

# Plot the subclusters again to show the change
sc.pl.embedding(erp_subset, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)
ax.set_title('ErP Subclustering - Updated Annotations', fontsize=12, fontweight='bold', y=1.1)

plt.show()

# Show updated distribution
print("\nUpdated distribution of consensus annotations within ErP subset:")
print(erp_subset.obs['Consensus_annotation_detailed'].value_counts())

# Check original annotations within each subcluster after reassignment
print("\nOriginal cell_type per subcluster (after reassignment):")
for cluster in sorted(erp_subset.obs['erp_subclusters'].unique()):
    cluster_cells = erp_subset.obs[erp_subset.obs['erp_subclusters'] == cluster]
    current_annotation = cluster_cells['Consensus_annotation_detailed'].iloc[0] if len(cluster_cells) > 0 else 'N/A'
    print(f"\nSubcluster {cluster} (now annotated as: {current_annotation}):")
    print(cluster_cells['cell_type'].value_counts())

# Update broad and simplified annotations for all reassigned cells
for new_annotation, cell_indices in reassigned_cells_summary.items():
    if len(cell_indices) > 0:
        if new_annotation == 'MPP':
            # MPP cells are immature HSPCs
            Luecken_dataset.obs.loc[cell_indices, 'Consensus_annotation_broad'] = 'Immature'
            Luecken_dataset.obs.loc[cell_indices, 'Consensus_annotation_simplified'] = 'HSPC'
        elif new_annotation == 'Macrophage':
            # Macrophages are mature monocytes
            Luecken_dataset.obs.loc[cell_indices, 'Consensus_annotation_broad'] = 'Mature'
            Luecken_dataset.obs.loc[cell_indices, 'Consensus_annotation_simplified'] = 'Macrophage'
        
        print(f"\nUpdated broad and simplified annotations for {len(cell_indices)} {new_annotation} cells")

# Show summary of all changes
print(f"\n" + "="*50)
print("SUMMARY OF ALL CHANGES:")
print("="*50)
print(f"- Subclusters 0 and 1: Kept as ErP")
for new_annotation, cell_indices in reassigned_cells_summary.items():
    count = len(cell_indices)
    if new_annotation == 'MPP':
        broad = 'Immature'
        simplified = 'HSPC'
    elif new_annotation == 'Macrophage':
        broad = 'Mature'
        simplified = 'Macrophage'
    
    print(f"- Reassigned {count} cells to {new_annotation}")
    print(f"  → Broad annotation: '{broad}'")
    print(f"  → Simplified annotation: '{simplified}'")
    print()

if total_reassigned == 0:
    print("No cells were reassigned.")

print(f"Total cells reassigned: {total_reassigned}")
print(f"ErP cells remaining: {(erp_subset.obs['Consensus_annotation_detailed'] == 'ErP').sum()}")

In [ ]:
# Plot UMAP with Consensus_annotation_detailed
sc.pl.embedding(Luecken_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(8, 6)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Merged datasets', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Show the figure
plt.show()

In [ ]:
# Remove unused categories
Luecken_dataset.obs['Consensus_annotation_simplified'] = Luecken_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Luecken_dataset.obs['Consensus_annotation_detailed'] = Luecken_dataset.obs['Consensus_annotation_detailed'].cat.remove_unused_categories()

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed_colors' in Luecken_dataset.uns:
    del Luecken_dataset.uns['Consensus_annotation_detailed_colors']

if 'Consensus_annotation_detailed_refined_colors' in Luecken_dataset.uns:
    del Luecken_dataset.uns['Consensus_annotation_detailed_refined_colors']

# Calculate silhouette scores for current annotations
print("Calculating silhouette scores for Luecken dataset...")

# Use the UMAP representation for silhouette analysis
X_embed = Luecken_dataset.obsm['X_umap']
labels = Luecken_dataset.obs['Consensus_annotation_detailed'].astype('category').cat.codes

# Calculate silhouette scores
silhouette_avg = silhouette_score(X_embed, labels)
sample_silhouette_values = silhouette_samples(X_embed, labels)

print(f"Average silhouette score: {silhouette_avg:.3f}")

# Add silhouette scores to the dataset
Luecken_dataset.obs['silhouette_score'] = sample_silhouette_values

# Identify cells with negative silhouette scores
negative_silhouette_mask = sample_silhouette_values < 0
print(f"Number of cells with negative silhouette scores: {negative_silhouette_mask.sum()}")
print(f"Percentage of cells with negative silhouette scores: {negative_silhouette_mask.sum()/len(sample_silhouette_values)*100:.2f}%")

# Show distribution of silhouette scores by cell type
silhouette_by_type = Luecken_dataset.obs.groupby('Consensus_annotation_detailed')['silhouette_score'].agg(['mean', 'std', 'min', 'max', 'count'])
print("\nSilhouette scores by cell type:")
print(silhouette_by_type.sort_values('mean'))

# Initialize refined annotations (start with original smoothed annotations)
Luecken_dataset.obs['Consensus_annotation_detailed_refined'] = Luecken_dataset.obs['Consensus_annotation_detailed'].copy()

# Perform silhouette-based reassignment
print("\n=== PERFORMING SILHOUETTE-BASED REASSIGNMENT ===")

# Identify cells with very poor silhouette scores (< -0.1)
very_poor_silhouette = Luecken_dataset.obs['silhouette_score'] < -0.1

if very_poor_silhouette.sum() > 0:
    print(f"Found {very_poor_silhouette.sum()} cells with very poor silhouette scores (< -0.1)")
    
    # Fit nearest neighbors
    nn = NearestNeighbors(n_neighbors=30, metric='euclidean')
    nn.fit(X_embed)
    
    # Get indices of poorly assigned cells
    poor_indices = np.where(very_poor_silhouette)[0]
    
    reassignments_made = 0
    
    for idx in poor_indices:
        # Find neighbors for this cell
        distances, neighbor_indices = nn.kneighbors([X_embed[idx]])
        neighbor_indices = neighbor_indices[0][1:]  # Exclude the cell itself
        
        # Get annotations of neighbors
        neighbor_annotations = Luecken_dataset.obs['Consensus_annotation_detailed'].iloc[neighbor_indices]

        # Find most common annotation among neighbors
        most_common = neighbor_annotations.mode()
        
        if len(most_common) > 0:
            new_annotation = most_common.iloc[0]
            current_annotation = Luecken_dataset.obs['Consensus_annotation_detailed'].iloc[idx]
            
            # Only reassign if the most common neighbor annotation is different
            if new_annotation != current_annotation:
                # Check if at least 40% of neighbors have this annotation
                fraction = (neighbor_annotations == new_annotation).sum() / len(neighbor_annotations)
                
                if fraction >= 0.4:
                    Luecken_dataset.obs.loc[Luecken_dataset.obs.index[idx], 'Consensus_annotation_detailed_refined'] = new_annotation
                    reassignments_made += 1
    
    print(f"Reassigned {reassignments_made} cells based on neighborhood consensus")
    
    # Recalculate silhouette scores after reassignment
    new_labels = Luecken_dataset.obs['Consensus_annotation_detailed_refined'].astype('category').cat.codes
    new_silhouette_scores = silhouette_samples(X_embed, new_labels)
    silhouette_avg_corrected = silhouette_score(X_embed, new_labels)
    
    # Store corrected scores
    Luecken_dataset.obs['silhouette_score_corrected'] = new_silhouette_scores
    
    print(f"\n=== REASSIGNMENT RESULTS ===")
    print(f"Original average silhouette: {silhouette_avg:.3f}")
    print(f"Refined average silhouette: {silhouette_avg_corrected:.3f}")
    print(f"Improvement: {silhouette_avg_corrected - silhouette_avg:.3f}")
    
    print(f"Original negative silhouette cells: {negative_silhouette_mask.sum()}")
    print(f"Refined negative silhouette cells: {(new_silhouette_scores < 0).sum()}")
    
    # Show what changes were made
    if reassignments_made > 0:
        changes_mask = (Luecken_dataset.obs['Consensus_annotation_detailed'] != 
                       Luecken_dataset.obs['Consensus_annotation_detailed_refined'])
        changes = Luecken_dataset.obs[changes_mask]
        
        print(f"\n=== SPECIFIC REASSIGNMENTS ===")
        change_summary = changes.groupby([
            'Consensus_annotation_detailed', 
            'Consensus_annotation_detailed_refined'
        ]).size().reset_index(name='count')
        
        for _, row in change_summary.iterrows():
            print(f"{row['Consensus_annotation_detailed']} -> {row['Consensus_annotation_detailed_refined']}: {row['count']} cells")

else:
    print("No cells with very poor silhouette scores found.")
    # Create corrected scores column that's identical to original
    Luecken_dataset.obs['silhouette_score_corrected'] = Luecken_dataset.obs['silhouette_score'].copy()
    silhouette_avg_corrected = silhouette_avg

# Create a reassignment status column for visualization
reassignment_mask = (Luecken_dataset.obs['Consensus_annotation_detailed'] != 
                    Luecken_dataset.obs['Consensus_annotation_detailed_refined'])
Luecken_dataset.obs['reassignment_status'] = 'Unchanged'
Luecken_dataset.obs.loc[reassignment_mask, 'reassignment_status'] = 'Reassigned'

# Final summary
print(f"\n=== FINAL SUMMARY ===")
print(f"Total cells: {len(Luecken_dataset)}")
print(f"Cells reassigned: {reassignment_mask.sum()}")
print(f"Final cell type distribution:")
final_counts = Luecken_dataset.obs['Consensus_annotation_detailed_refined'].value_counts()
print(final_counts)

# Clear ALL color palettes before plotting to ensure fresh colors
color_keys_to_clear = [
    'Consensus_annotation_detailed_colors',
    'Consensus_annotation_detailed_refined_colors',
    'reassignment_status_colors',
    'silhouette_score_corrected_colors'
]

for key in color_keys_to_clear:
    if key in Luecken_dataset.uns:
        del Luecken_dataset.uns[key]

# Plot comprehensive analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Original annotations
sc.pl.embedding(Luecken_dataset, 
                color='Consensus_annotation_detailed', 
                basis='X_umap',
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,0])
axes[0,0].set_title('Original Smoothed Annotations', fontsize=14, fontweight='bold')

# Plot 2: Refined annotations
sc.pl.embedding(Luecken_dataset, 
                color='Consensus_annotation_detailed_refined', 
                basis='X_umap',
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,1])
axes[0,1].set_title('Silhouette-Refined Annotations', fontsize=14, fontweight='bold')

# Plot 3: Reassignment status
sc.pl.embedding(Luecken_dataset, 
                color='reassignment_status', 
                basis='X_umap',
                palette={'Unchanged': 'lightgray', 'Reassigned': 'red'},
                add_outline=False,
                legend_loc='right margin', 
                frameon=False,
                show=False,
                ax=axes[1,0])
axes[1,0].set_title('Reassignment Status', fontsize=14, fontweight='bold')

# Plot 4: Corrected silhouette scores
sc.pl.embedding(Luecken_dataset, 
                color='silhouette_score_corrected', 
                basis='X_umap',
                color_map='RdBu_r',
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[1,1])
axes[1,1].set_title('Silhouette Scores (Refined)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(figures_path + "/Luecken_dataset_silhouette_refinement_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

# Additional histogram comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# Original silhouette distribution
ax1.hist(sample_silhouette_values, bins=50, alpha=0.7, edgecolor='black', color='lightblue')
ax1.axvline(x=0, color='red', linestyle='--', label='Silhouette = 0')
ax1.set_xlabel('Silhouette Score')
ax1.set_ylabel('Number of Cells')
ax1.set_title(f'Original Silhouette Distribution\n(Avg: {silhouette_avg:.3f})')
ax1.legend()

# Refined silhouette distribution
ax2.hist(Luecken_dataset.obs['silhouette_score_corrected'], bins=50, alpha=0.7, edgecolor='black', color='lightgreen')
ax2.axvline(x=0, color='red', linestyle='--', label='Silhouette = 0')
ax2.set_xlabel('Silhouette Score')
ax2.set_ylabel('Number of Cells')
ax2.set_title(f'Refined Silhouette Distribution\n(Avg: {silhouette_avg_corrected:.3f})')
ax2.legend()

plt.tight_layout()
plt.savefig(figures_path + "/Luecken_dataset_silhouette_distribution_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Remove unused categories
Luecken_dataset.obs['Consensus_annotation_simplified'] = Luecken_dataset.obs['Consensus_annotation_simplified'].cat.remove_unused_categories()
Luecken_dataset.obs['Consensus_annotation_detailed_refined'] = Luecken_dataset.obs['Consensus_annotation_detailed_refined'].cat.remove_unused_categories()

In [ ]:
Luecken_dataset_normalized = Luecken_dataset.copy()
Luecken_dataset_normalized.X = SCUtils.Protein_normalization(Luecken_dataset_normalized.X)
sc.tl.rank_genes_groups(Luecken_dataset_normalized, 'Consensus_annotation_detailed_refined', method='wilcoxon')
sc.pl.rank_genes_groups(Luecken_dataset_normalized, n_genes=10, sharey=False, ncols = 3, fontsize = 14)

plt.savefig(figures_path + "/Luecken_dataset_top10_markers.png", dpi=300, bbox_inches='tight')

In [ ]:
AveragedExpression = grouped_obs_mean(Luecken_dataset_normalized, 'Consensus_annotation_detailed_refined')
df = pd.DataFrame(AveragedExpression)

In [ ]:
# Compute the correlation matrix
corr = df.corr(method='pearson')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(235, 15, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
heatmap = sns.heatmap(corr, mask=mask, cmap=cmap, annot=True, 
                        square=True, linewidths=.6, cbar_kws={"shrink": 1},
                        annot_kws={"fontsize":5})

heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12)

plt.savefig(figures_path + "/Luecken_dataset_correlation_heatmap.png", dpi=300, bbox_inches='tight')

In [ ]:
Luecken_dataset.obs['Consensus_annotation_broad_final'] = 'Mature'

categories = ['Immature','Mature']

Luecken_dataset.obs['Consensus_annotation_broad_final'] = pd.Categorical(Luecken_dataset.obs['Consensus_annotation_broad_final'], categories=categories)
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['MPP', 'Pro-B']), 'Consensus_annotation_broad_final'] = 'Immature'

In [ ]:
Luecken_dataset.obs['Consensus_annotation_simplified_final'] = ''

categories = ['HSPC', 'Monocyte', 'CD4 T', 'CD8 T', 'Erythroid', 'B', 'cDC', 'pDC', 'NK', 'Macrophage', 'ILC', 'Stroma', 'Myeloid', 'Other T', 'Plasma']

Luecken_dataset.obs['Consensus_annotation_simplified_final'] = pd.Categorical(Luecken_dataset.obs['Consensus_annotation_simplified_final'], categories=categories)
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['MPP', 'Pro-B']), 'Consensus_annotation_simplified_final'] = 'HSPC'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD14 Mono', 'CD16 Mono']), 'Consensus_annotation_simplified_final'] = 'Monocyte'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['NK CD56 dim', 'NK CD56 bright']), 'Consensus_annotation_simplified_final'] = 'NK'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD4 T Naive', 'CD4 T Memory', 'Treg', 'CD4 CTL']), 'Consensus_annotation_simplified_final'] = 'CD4 T'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['CD8 T Naive', 'CD8 T Memory', 'MAIT']), 'Consensus_annotation_simplified_final'] = 'CD8 T'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['B Naive', 'B Memory', 'Plasma', 'Immature B', 'Pre-B']), 'Consensus_annotation_simplified_final'] = 'B'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Erythroblast', 'ErP']), 'Consensus_annotation_simplified_final'] = 'Erythroid'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'] == 'Myeloid progenitor', 'Consensus_annotation_simplified_final'] = 'Myeloid'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['cDC1', 'cDC2']), 'Consensus_annotation_simplified_final'] = 'cDC'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'] == 'pDC', 'Consensus_annotation_simplified_final'] = 'pDC'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'].isin(['Double negative T', 'Gamma delta T']), 'Consensus_annotation_simplified_final'] = 'Other T'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'] == 'Macrophage', 'Consensus_annotation_simplified_final'] = 'Macrophage'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'] == 'ILC', 'Consensus_annotation_simplified_final'] = 'ILC'
Luecken_dataset.obs.loc[Luecken_dataset.obs['Consensus_annotation_detailed_refined'] == 'Plasma', 'Consensus_annotation_simplified_final'] = 'Plasma'

In [ ]:
Luecken_dataset.obs['Consensus_annotation_detailed_final'] = Luecken_dataset.obs['Consensus_annotation_detailed_refined']

In [ ]:
# Remove unused categories
Luecken_dataset.obs['Consensus_annotation_simplified_final'] = Luecken_dataset.obs['Consensus_annotation_simplified_final'].cat.remove_unused_categories()
Luecken_dataset.obs['Consensus_annotation_detailed_final'] = Luecken_dataset.obs['Consensus_annotation_detailed_final'].cat.remove_unused_categories()

In [ ]:
# Plot UMAP with color
sc.pl.embedding(Luecken_dataset, 
                color='Consensus_annotation_broad_final', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Luecken M.D. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus broad annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Luecken_dataset_final_consensus_annotation_broad_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_simplified_final_colors' in Luecken_dataset.uns:
    del Luecken_dataset.uns['Consensus_annotation_simplified_final_colors']

# Plot UMAP with color
sc.pl.embedding(Luecken_dataset, 
                color='Consensus_annotation_simplified_final', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Luecken M.D. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus simplified annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Luecken_dataset_final_consensus_annotation_simplified_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

In [ ]:
# Clear any existing color palettes to force scanpy to regenerate them
if 'Consensus_annotation_detailed_final_colors' in Luecken_dataset.uns:
    del Luecken_dataset.uns['Consensus_annotation_detailed_final_colors']

# Plot UMAP with color
sc.pl.embedding(Luecken_dataset, 
                color='Consensus_annotation_detailed_final', 
                basis='X_umap', 
                legend_loc='on data', 
                legend_fontsize=5,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

# Get the current axis and set axis labels and tick labels
ax = plt.gca()
ax.figure.set_size_inches(6, 5)
ax.set_xlabel('UMAP 1', fontsize=12)
ax.set_ylabel('UMAP 2', fontsize=12)

# Set the title with font size 14, bold, and increased distance from the plot
ax.set_title('Luecken M.D. et al. dataset', fontsize=12, fontweight='bold', y=1.1)

# Add a subtitle
plt.suptitle('Consensus detailed annotation', fontsize=8, y=0.925, color=(0.5, 0.5, 0.5))

# Save the figure at 300 dpi
plt.savefig(figures_path + "/Luecken_dataset_final_Consensus_annotation_detailed_annotation.png", 
            dpi=300, bbox_inches='tight')

# Show the figure
plt.show()

# Writing results

In [ ]:
Zhang_dataset.write_h5ad(data_path + "/References/Zhang" + "/Zhang_adata_annotated.h5ad")
Hao_dataset.write_h5ad(data_path + "/References/Hao" + "/228AB_healthy_donors_PBMNCs_annotated.h5ad")
Triana_dataset.write_h5ad(data_path + "/References/Triana" + "/97AB_young_and_old_adult_healthy_donor_BMMNCs_annotated.h5ad")
Luecken_dataset.write_h5ad(data_path + "/References/Luecken" + "/140AB_adult_healthy_donor_BMMNCs_annotated.h5ad")

In [ ]:
# Get the remaining cell barcodes from all four processed datasets
print("Getting remaining cell barcodes from all processed datasets...")

remaining_barcodes = set()
remaining_barcodes.update(Zhang_dataset.obs_names)
remaining_barcodes.update(Hao_dataset.obs_names)
remaining_barcodes.update(Triana_dataset.obs_names)
remaining_barcodes.update(Luecken_dataset.obs_names)

print(f"Total remaining cells across all datasets: {len(remaining_barcodes)}")

# Filter the original adatas object to keep only remaining cells
print("Filtering original merged adatas object...")
remaining_mask = adatas.obs_names.isin(remaining_barcodes)
adatas_final = adatas[remaining_mask].copy()

print(f"Original adatas shape: {adatas.shape}")
print(f"Filtered adatas_final shape: {adatas_final.shape}")
print(f"Cells removed: {adatas.n_obs - adatas_final.n_obs}")

# Check which cells remain from each dataset
print(f"\nCells remaining per dataset:")
print(adatas_final.obs['dataset_name'].value_counts())

# Initialize final annotation columns
adatas_final.obs['Consensus_annotation_detailed_final'] = ''
adatas_final.obs['Consensus_annotation_simplified_final'] = ''
adatas_final.obs['Consensus_annotation_broad_final'] = ''

# Assign final annotations from each processed dataset
print("\nAssigning final consensus annotations...")

# Zhang dataset assignments
zhang_mask = adatas_final.obs['dataset_name'] == 'Zhang'
zhang_indices = adatas_final.obs_names[zhang_mask]
zhang_overlap = zhang_indices.intersection(Zhang_dataset.obs_names)

if len(zhang_overlap) > 0:
    adatas_final.obs.loc[zhang_overlap, 'Consensus_annotation_detailed_final'] = Zhang_dataset.obs.loc[zhang_overlap, 'Consensus_annotation_detailed_final'].values
    adatas_final.obs.loc[zhang_overlap, 'Consensus_annotation_simplified_final'] = Zhang_dataset.obs.loc[zhang_overlap, 'Consensus_annotation_simplified_final'].values
    adatas_final.obs.loc[zhang_overlap, 'Consensus_annotation_broad_final'] = Zhang_dataset.obs.loc[zhang_overlap, 'Consensus_annotation_broad_final'].values
    print(f"Zhang: Assigned annotations to {len(zhang_overlap)} cells")

# Hao dataset assignments
hao_mask = adatas_final.obs['dataset_name'] == 'Hao'
hao_indices = adatas_final.obs_names[hao_mask]
hao_overlap = hao_indices.intersection(Hao_dataset.obs_names)

if len(hao_overlap) > 0:
    adatas_final.obs.loc[hao_overlap, 'Consensus_annotation_detailed_final'] = Hao_dataset.obs.loc[hao_overlap, 'Consensus_annotation_detailed_final'].values
    adatas_final.obs.loc[hao_overlap, 'Consensus_annotation_simplified_final'] = Hao_dataset.obs.loc[hao_overlap, 'Consensus_annotation_simplified_final'].values
    adatas_final.obs.loc[hao_overlap, 'Consensus_annotation_broad_final'] = Hao_dataset.obs.loc[hao_overlap, 'Consensus_annotation_broad_final'].values
    print(f"Hao: Assigned annotations to {len(hao_overlap)} cells")

# Triana dataset assignments
triana_mask = adatas_final.obs['dataset_name'] == 'Triana'
triana_indices = adatas_final.obs_names[triana_mask]
triana_overlap = triana_indices.intersection(Triana_dataset.obs_names)

if len(triana_overlap) > 0:
    adatas_final.obs.loc[triana_overlap, 'Consensus_annotation_detailed_final'] = Triana_dataset.obs.loc[triana_overlap, 'Consensus_annotation_detailed_final'].values
    adatas_final.obs.loc[triana_overlap, 'Consensus_annotation_simplified_final'] = Triana_dataset.obs.loc[triana_overlap, 'Consensus_annotation_simplified_final'].values
    adatas_final.obs.loc[triana_overlap, 'Consensus_annotation_broad_final'] = Triana_dataset.obs.loc[triana_overlap, 'Consensus_annotation_broad_final'].values
    print(f"Triana: Assigned annotations to {len(triana_overlap)} cells")

# Luecken dataset assignments
luecken_mask = adatas_final.obs['dataset_name'] == 'Luecken'
luecken_indices = adatas_final.obs_names[luecken_mask]
luecken_overlap = luecken_indices.intersection(Luecken_dataset.obs_names)

if len(luecken_overlap) > 0:
    adatas_final.obs.loc[luecken_overlap, 'Consensus_annotation_detailed_final'] = Luecken_dataset.obs.loc[luecken_overlap, 'Consensus_annotation_detailed_final'].values
    adatas_final.obs.loc[luecken_overlap, 'Consensus_annotation_simplified_final'] = Luecken_dataset.obs.loc[luecken_overlap, 'Consensus_annotation_simplified_final'].values
    adatas_final.obs.loc[luecken_overlap, 'Consensus_annotation_broad_final'] = Luecken_dataset.obs.loc[luecken_overlap, 'Consensus_annotation_broad_final'].values
    print(f"Luecken: Assigned annotations to {len(luecken_overlap)} cells")

# Convert to categorical
adatas_final.obs['Consensus_annotation_detailed_final'] = pd.Categorical(adatas_final.obs['Consensus_annotation_detailed_final'])
adatas_final.obs['Consensus_annotation_simplified_final'] = pd.Categorical(adatas_final.obs['Consensus_annotation_simplified_final'])
adatas_final.obs['Consensus_annotation_broad_final'] = pd.Categorical(adatas_final.obs['Consensus_annotation_broad_final'])

# Remove any cells that didn't get annotations assigned (shouldn't happen but safety check)
unassigned_mask = adatas_final.obs['Consensus_annotation_detailed_final'] == ''
if unassigned_mask.sum() > 0:
    print(f"Warning: {unassigned_mask.sum()} cells did not receive final annotations. Removing them.")
    adatas_final = adatas_final[~unassigned_mask]

print(f"\n=== FINAL DATASET SUMMARY ===")
print(f"Final dataset shape: {adatas_final.shape}")
print(f"Final cells per dataset:")
print(adatas_final.obs['dataset_name'].value_counts())

print(f"\nFinal broad annotation distribution:")
print(adatas_final.obs['Consensus_annotation_broad_final'].value_counts())

print(f"\nFinal simplified annotation distribution:")
print(adatas_final.obs['Consensus_annotation_simplified_final'].value_counts())

print(f"\nFinal detailed annotation distribution:")
print(adatas_final.obs['Consensus_annotation_detailed_final'].value_counts())

# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Plot 1: Dataset distribution
sc.pl.embedding(adatas_final, 
                color='dataset_name', 
                basis='X_umap', 
                legend_loc='right margin',
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,0])
axes[0,0].set_title('Dataset Distribution', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('UMAP 1', fontsize=12)
axes[0,0].set_ylabel('UMAP 2', fontsize=12)

# Plot 2: Final broad annotations
sc.pl.embedding(adatas_final, 
                color='Consensus_annotation_broad_final', 
                basis='X_umap',
                legend_loc='right margin',
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,1])
axes[0,1].set_title('Final Broad Annotations', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('UMAP 1', fontsize=12)
axes[0,1].set_ylabel('UMAP 2', fontsize=12)

# Plot 3: Final simplified annotations
sc.pl.embedding(adatas_final, 
                color='Consensus_annotation_simplified_final', 
                basis='X_umap',
                legend_loc='on data',
                legend_fontsize=6,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[0,2])
axes[0,2].set_title('Final Simplified Annotations', fontsize=14, fontweight='bold')
axes[0,2].set_xlabel('UMAP 1', fontsize=12)
axes[0,2].set_ylabel('UMAP 2', fontsize=12)

# Plot 4: Final detailed annotations
sc.pl.embedding(adatas_final, 
                color='Consensus_annotation_detailed_final', 
                basis='X_umap',
                legend_loc='on data',
                legend_fontsize=4,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[1,0])
axes[1,0].set_title('Final Detailed Annotations', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('UMAP 1', fontsize=12)
axes[1,0].set_ylabel('UMAP 2', fontsize=12)

# Plot 5: Chemistry distribution
sc.pl.embedding(adatas_final, 
                color='Chemistry', 
                basis='X_umap',
                legend_loc='right margin',
                add_outline=False,
                frameon=False,
                show=False,
                ax=axes[1,1])
axes[1,1].set_title('Chemistry Distribution', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('UMAP 1', fontsize=12)
axes[1,1].set_ylabel('UMAP 2', fontsize=12)

# Plot 6: Summary statistics as text
axes[1,2].axis('off')
summary_text = f"""
Final Dataset Summary:

Total cells: {len(adatas_final):,}
Total features: {adatas_final.n_vars}

Cells per dataset:
Zhang: {(adatas_final.obs['dataset_name'] == 'Zhang').sum():,}
Hao: {(adatas_final.obs['dataset_name'] == 'Hao').sum():,}
Triana: {(adatas_final.obs['dataset_name'] == 'Triana').sum():,}
Luecken: {(adatas_final.obs['dataset_name'] == 'Luecken').sum():,}

Broad categories:
{chr(10).join([f"{cat}: {count:,}" for cat, count in adatas_final.obs['Consensus_annotation_broad_final'].value_counts().items()])}

Cell types identified:
Detailed: {adatas_final.obs['Consensus_annotation_detailed_final'].nunique()}
Simplified: {adatas_final.obs['Consensus_annotation_simplified_final'].nunique()}
"""

axes[1,2].text(0.05, 0.95, summary_text, transform=axes[1,2].transAxes, 
               fontsize=10, verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.savefig(figures_path + "/Final_merged_datasets_comprehensive_overview.png", 
            dpi=300, bbox_inches='tight')
plt.show()

print("\nFinal merged dataset with consensus annotations is ready!")

In [ ]:
sc.pl.embedding(adatas_final, 
                color='Consensus_annotation_detailed_final', 
                basis='X_umap',
                legend_loc='on data',
                legend_fontsize=4,
                legend_fontoutline=2,
                add_outline=False,
                frameon=False,
                show=False)

plt.tight_layout()
plt.show()
